This notebook takes a look at the scrapped results from the BoardDocs website and gets the correct addresses for each row.

Input:
- `prelim_results.csv`

In [1600]:
import pandas as pd

df = pd.read_csv("prelim_results.csv")
df.head()

,URL,title_1,title_2,home_website
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,Board Policies and Guidelines,St. Joseph Public Schools,https://www.sjschools.org/
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,School Board Policy Manual,NaN,www.calsd.org
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",http://www.mapleschools.com
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District,https://www.riversidelocalschools.com/
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,School Board Policy Manual,Southern Huntingdon County School District,http://www.shcsd.org


In [1601]:
# we remove redundant info
df.loc[df["title_1"]==df["title_2"], "title_1"] = None

In [1602]:
# trim whitespace
df.loc[:, "title_1"] = df["title_1"].str.strip()
df.loc[:, "title_2"] = df["title_2"].str.strip()

In [1603]:
# consider the 5-digit zip code approach
# check which rows have 5-digit codes in Title1, and those with them in Title2.
# Hopefully this will be a partition

# Define a regex pattern to match a 5-digit zip code
zip_code_pattern = r'\b\d{5}\b'

# Find rows where Title1 has a 5-digit zip code
df['Title1_has_zipcode'] = df['title_1'].str.contains(zip_code_pattern, na=False)

# Find rows where Title2 has a 5-digit zip code
df['Title2_has_zipcode'] = df['title_2'].str.contains(zip_code_pattern, na=False)

In [1604]:
# check if it is a partition
# first check if they add up

import numpy as np

print(f"Num of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()}")
print(f"Num of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()}")
print(f"Num of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])}")
print(f"Num of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode'])}")
print(f"Num of total rows {df.shape[0]}")

Num of rows where title 1 has zipcode 1440
Num of rows where title 2 has zipcode 1442
Num of rows where title 1 or title 2 has zipcode 2882
Num of rows where title 1 and title 2 have zipcode 0
Num of total rows 3906


In [1605]:
# ok great, there are no rows where you can find zipcodes on both cols
# but some rows don't have zipcodes in either
# let's check them out

no_zipcode_df = df[~(df['Title1_has_zipcode'] | df['Title2_has_zipcode'])]
print(f"Num of rows without zipcode {no_zipcode_df.shape[0]}")
no_zipcode_df.sample(5)

Num of rows without zipcode 1024


,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,False,False
3005,https://go.boarddocs.com/oh/franklinmonroe/Board.nsf/Public,NaN,NaN,http://www.franklin-monroe.k12.oh.us,False,False
3404,https://go.boarddocs.com/in/stoh/Board.nsf/Public,School Town of Highland,School Board Policies,http://www.highland.k12.in.us,False,False
383,https://go.boarddocs.com/mi/aops/Board.nsf/Public,School Board Policies,AuTrain-Onota Public Schools,http://www.autrainonota.com,False,False
3509,https://go.boarddocs.com/in/npusc/Board.nsf/Public,School Board Policies and Guidelines,New Prairie United School Corporation,http://www.npusc.k12.in.us,False,False


In [1606]:
# let's get the proportions


print(f"Percentage of rows where title 1 has zipcode {df['Title1_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 2 has zipcode {df['Title2_has_zipcode'].sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 or title 2 has zipcode {np.sum(df['Title1_has_zipcode'] | df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows where title 1 and title 2 have zipcode {np.sum(df['Title1_has_zipcode'] & df['Title2_has_zipcode']).sum()/df.shape[0]*100:.2f}%")
print(f"Percentage of rows with no zipcodes {no_zipcode_df.shape[0]/df.shape[0]*100:.2f}%")


Percentage of rows where title 1 has zipcode 36.87%
Percentage of rows where title 2 has zipcode 36.92%
Percentage of rows where title 1 or title 2 has zipcode 73.78%
Percentage of rows where title 1 and title 2 have zipcode 0.00%
Percentage of rows with no zipcodes 26.22%


In [1607]:
# some NaNs, some "Policy Manual", "School Board Policy and Guidelines", etc
# let's check the most common values

no_zipcode_df["title_1"].value_counts().head()

title_1
School Board Policy Manual              115
Policy Manual                            54
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Name: count, dtype: int64

In [1608]:
no_zipcode_df["title_2"].value_counts().head()

title_2
Board Policies                 24
                               22
School Board Policies          17
Board of Education             11
Board of Education Policies    11
Name: count, dtype: int64

In [1609]:
# ok, now let's check the website col
sum(df["home_website"].isna())

53

In [1610]:
# ok, unfortunately there are boarddocs without the home website linked
print(f"Percent of websites without links to official: {sum(df["home_website"].isna())/df.shape[0]*100:.3}%")

Percent of websites without links to official: 1.36%


In [1611]:
# but thankfully this number is small
# let's take a look at these websites

# do not truncate the col values in display
pd.set_option('display.max_colwidth', None)

df[df["home_website"].isna()].head()

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
232,https://go.boarddocs.com/pa/camb/Board.nsf/Public,NaN,NaN,NaN,False,False
443,https://go.boarddocs.com/oh/oakhil/Board.nsf/Public,NaN,NaN,NaN,False,False
465,https://go.boarddocs.com/oh/warrenoh/Board.nsf/Public,NaN,NaN,NaN,False,False
468,https://go.boarddocs.com/pa/marp/Board.nsf/Public,School Board Policy Manual,Marple Newtown School District,NaN,False,False
533,https://go.boarddocs.com/oh/meigs/Board.nsf/Public,NaN,NaN,NaN,False,False


In [1612]:
# after inspecting a few, it seems like they will usually write their school district as the h1 tag at least.

In [1613]:
# back to the address
# an observation is that I don't think those that don't contain zip codes will have addresses on the website
# let's check if a single number exists in them

number_pattern = r'\d'
print("Number of no-zipcode rows that contain a number")
sum(no_zipcode_df["title_1"].str.contains(number_pattern, na=False))

Number of no-zipcode rows that contain a number


46

In [1614]:
no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
919,https://go.boarddocs.com/wi/slj1/Board.nsf/Public,Joint School District No 1 Silver Lake - Salem,School Board Policies and Guidelines,https://www.silverlakejt1.k12.wi.us,False,False
2983,https://go.boarddocs.com/il/rlasd/Board.nsf/Public,Round Lake Area School District CUSD 116,NaN,http://www.rlas-116.org,False,False
3156,https://go.boarddocs.com/il/elmhurst/Board.nsf/Public,Elmhurst Community Unit School Dist. 205,eGovernance Site,http://www.elmhurst205.org,False,False
1987,https://go.boarddocs.com/oh/nls/Board.nsf/Public,600 Lemoyne Rd | Northwood OH | P: (419) 691-3888 | F: (419) 697-2470,Northwood Local Schools,http://www.northwoodschools.org/site/default.aspx?PageID=1,False,False
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,Napoleon Area City School District,www.napoleon.k12.oh.us,False,False


In [1615]:
no_zipcode_df[no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode
479,https://go.boarddocs.com/pa/iu06/Board.nsf/Public,"Meetings, Agendas, Policy Manual",Riverview Intermediate Unit 6,http://www.riu6.org,False,False
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,False,False
3500,https://go.boarddocs.com/mi/carain/Board.nsf/Public,Carman-Ainsworth Community Schools,810-591-3700,http://www.carmanainsworth.org,False,False
3249,https://go.boarddocs.com/pa/iu27/Board.nsf/Public,Policy Manual,Beaver Valley Intermediate Unit 27,http://www.bviu.org,False,False
155,https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public,Clinton Central School District,75 Chenango Ave Clinton NY13323,https://www.ccs.edu/,False,False


In [1616]:
# it turns out that they could either be an address with a missing zip code, or phone numbers
# there are also misc cases

# let's get a conservative (high) bound on the number of addresses that we will miss

num_no_zipcode_with_number = no_zipcode_df[no_zipcode_df["title_1"].str.contains(number_pattern, na=False) | no_zipcode_df["title_2"].str.contains(number_pattern, na=False)].shape[0]
num_no_zipcode_with_number

105

In [1617]:
num_zipcode = df.shape[0] - no_zipcode_df.shape[0]

In [1618]:
print(f"Worst case proportion of addresses that we will miss {num_no_zipcode_with_number/(num_no_zipcode_with_number+num_zipcode)*100:.2}%")

Worst case proportion of addresses that we will miss 3.5%


In [1619]:
# we put the ones we know are correct at a new address field

df["address"] = None
df.loc[df['Title1_has_zipcode'], "address"] = df["title_1"]
df.loc[df['Title2_has_zipcode'], "address"] = df["title_2"]
df.sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address
3360,https://go.boarddocs.com/mi/mmaec/Board.nsf/Public,Muskegon Montessori Academy for Environmental Change,"2950 McCracken St., Norton Shores, MI 49441 Phone: 231-766-7500",https://muskegonmontessoriacademy.com/,False,True,"2950 McCracken St., Norton Shores, MI 49441 Phone: 231-766-7500"
2894,https://go.boarddocs.com/oh/streetsboro/Board.nsf/Public,"9000 Kirby Lane | Streetsboro, OH 44241 | P 330-626-4900 | F 330-626-8102",Streetsboro City Schools,http://www.scsrockets.org/,True,False,"9000 Kirby Lane | Streetsboro, OH 44241 | P 330-626-4900 | F 330-626-8102"
745,https://go.boarddocs.com/pa/carl/Board.nsf/Public,Carlynton School District,"435 Kings Highway Carnegie, PA 15106",http://www.carlynton.k12.pa.us/,False,True,"435 Kings Highway Carnegie, PA 15106"
1288,https://go.boarddocs.com/wi/wauw/Board.nsf/Public,"12121 W. North Ave. | Wauwatosa, WI 53226 | (414) 773-1000",Wauwatosa School District,https://www.boarddocs.com/wi/wauw/Board.nsf,True,False,"12121 W. North Ave. | Wauwatosa, WI 53226 | (414) 773-1000"
3159,https://go.boarddocs.com/wi/wawmsd/Board.nsf/Public,"West Allis-West Milwaukee School District, 9333 W. Lincoln Avenue, West Allis, WI 53227",414-604-3000,http://www.wawmsd.org,True,False,"West Allis-West Milwaukee School District, 9333 W. Lincoln Avenue, West Allis, WI 53227"


In [1620]:
# now, let's try to get the school district name.
# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

In [1621]:
# number of rows with schools in at least one col
df[df["title_1_has_school"] | df["title_2_has_school"]].shape[0]

3206

In [1622]:
# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

356

In [1623]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County Schools,Jefferson County School Board,https://www.jeffersonschools.net,False,False,None,True,True
2936,https://go.boarddocs.com/in/scsd1/Board.nsf/Public,School Board Policies and Guidelines,Scott County School District #1,http://www.scsd1.com,False,False,None,True,True
2346,https://go.boarddocs.com/oh/bell/Board.nsf/Public,School Board Policies and Guidelines,Bellefontaine City Schools,NaN,False,False,None,True,True
3087,https://go.boarddocs.com/oh/hopewell/Board.nsf/Public,School Board Policies,Hopewell-Loudon Local Schools,www.hlschool.org,False,False,None,True,True
262,https://go.boarddocs.com/oh/minls/Board.nsf/Public,School District Policies,Minford Local Schools,www.minford.k12.oh.us,False,False,None,True,True


In [1624]:
# there is boilerplate text like School Board Policies
# let's try to get the top few popular ones and remove them

df["title_1"].value_counts().head(10)

title_1
School Board Policy Manual              118
Policy Manual                            55
School Board Policies and Guidelines     41
School Board Policies                    22
BoardDocs PL                              9
Board Policy and Guidelines               8
Board Policies                            7
School Board Policies and Bylaws          7
School Board Policies & Bylaws            6
Board Policies and Guidelines             6
Name: count, dtype: int64

In [1625]:
df["title_2"].value_counts().head(10)

title_2
Board Policies                      25
                                    25
School Board Policies               18
Board of Education                  14
Board of Education Policies         12
Board of Education Policy Manual    10
NEOLA Board Policies                10
Policy Manual                        6
School Board Policy Manual           6
Board Policy Manual                  5
Name: count, dtype: int64

In [1626]:
# let's remove these

remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board Policy                          6
School District Policies              6
Board Policy Manual                   5
Board Policies and Bylaws             5
School Board Policies & Guidelines    4
eGovernance Site                      4
School Board Policy                   4
School District Policy Manual         3
Board Policies & Bylaws               3
Board Policy and Bylaws               3
Name: count, dtype: int64

In [1627]:
# let's remove these
remove_title_1_list = df["title_1"].value_counts().head(10).index
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
Board of Education Policies and Guidelines                                                  3
School Board Policy and Guidelines                                                          3
School Board Policies and Administrative Guidelines                                         3
                                                                                            3
Board of Education Policies                                                                 3
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Name: count, dtype: int64

In [1628]:
# let's remove the first few
remove_title_1_list = df["title_1"].value_counts().head(5).index
remove_title_1_list

Index(['Board of Education Policies and Guidelines',
       'School Board Policy and Guidelines',
       'School Board Policies and Administrative Guidelines', '',
       'Board of Education Policies'],
      dtype='object', name='title_1')

In [1629]:
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
df["title_1"].value_counts().head(10)

title_1
2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180    2
1300 Sherman Street Ste 222 | Sturgis, SD  57785                                            2
Bloomfield School District                                                                  2
1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500                           2
School Board Policy & Guidelines                                                            2
1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009           2
Electronic Governance System                                                                2
Policy Manual and Agendas                                                                   2
Board of Education                                                                          2
60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700                           2
Name: count, dtype: int64

In [1630]:
# pick those
remove_title_1_list = [
    "School Board By-Laws and Policies",
    "School Board policies and guidelines",
    "Policy Manual and Agendas",
    "Board of Education Policies",
    "School Board Policy and Bylaws",
    "Board Policy and Bylaws",
    "Board Policies and By-Laws",
    "School Board Policies ",
    "Board of Education",
    "School Board Policies and Guidlines",
    "School Board Bylaws and Policies",
    "SCHOOL BOARD POLICIES",
    "Board Policies & Guidelines",
    "School Board Policy & Guidelines",
    "Electronic Governance System",
    "School Board Agendas and Minutes",
    "School Board Policy and Administrative Guidelines",
    "School District Policies / Policy Manual",
    "School Policy Manual",
    "School Board Meetings",
    "School Board Policies & Ad Guidelines",
    "School Board meetings, agendas and policies",
    "School Board Policies and By Laws",
    "School board policies and manuals",
    "Board of School Directors",
    "School Board Agendas, Minutes, Policies and Guidelines",
    "Board Approved School Policy",
    "School Board Policies & Administrative Guidelines",
    "School Board Policy",
    "School Board Agendas, Policies and Guidelines",
    "School Board Bylaws/Policies/Guidelines",
    "SchoolBoard Policy Manual",
    "School Board Policy and Guidlines",
    "School Board Policy",
    "Governing Board Policy",
    "Board policy and guidelines",
    "Policies & By Laws",
    "Board of Trustees Policies and Administrative Guidelines",
    "Board of Education Policy and Bylaws"
    
]
df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None
# let's check what's left
print(df["title_1"].value_counts().index[:10])

Index(['2680 West County Road 476 | Bushnell, Florida 33513 | Ph: 352-793-2315  Fx: 352-793-4180',
       '200 Reid Street | Palatka, FL 32177 | (386) 329-0602',
       '60 Jefferson Street, Suite 3• Monticello, NY 12701 • 845-794-7700',
       'Bloomfield School District',
       '1290 Ridder Park Drive | San Jose, CA 95131-2304 | (408) 453-6500',
       '1300 Sherman Street Ste 222 | Sturgis, SD  57785',
       '315 N. French Avenue | Arlington  WA 98223 | 360.618.6200 | f 360.618.6221',
       '1725 North Dodge St. |  Iowa City, IA 52245 | p (319) 688-1000 | f (319) 688-1009',
       'Cleveland Metropolitan School District',
       '6301 Springside Avenue | Downers Grove, IL 60516 | Ph: (630) 795-7100  | Fx: (630) 795-7199'],
      dtype='object', name='title_1')


In [1631]:
# do this for title_2
# let's remove these
# NOTE: NEOLOA Board Policies might provide coarse information on whether the school could be located

remove_title_2_list = df["title_2"].value_counts().head(10).index
remove_title_2_list

Index(['Board Policies', '', 'School Board Policies', 'Board of Education',
       'Board of Education Policies', 'Board of Education Policy Manual',
       'NEOLA Board Policies', 'Policy Manual', 'School Board Policy Manual',
       'Board Policy Manual'],
      dtype='object', name='title_2')

In [1632]:
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
eGovernance Site                 4
Bylaws & Policies                4
Board of Education Policy        4
Board of Education Meetings      3
Board Policy                     3
Neola Board Policies             3
Board of Directors               3
Arlington Public Schools         3
Putnam County School District    2
BoardDocs - Meeting Agendas      2
Name: count, dtype: int64

In [1633]:
remove_title_2_list = df["title_2"].value_counts().head(4).index
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().head(10)

title_2
Arlington Public Schools                  3
Neola Board Policies                      3
Board of Directors                        3
Board Policy                              3
Board Agendas                             2
Monticello Central School District        2
Meetings, Agendas and Information         2
Green Local Schools                       2
Santa Clara County Office of Education    2
Meade County, South Dakota                2
Name: count, dtype: int64

In [1634]:
remove_title_2_list = [
    "Meetings and Information",
    "Meetings, Agendas and Information",
    "Board Policy Manual",
    "Board of Education Meetings",
    "Board of Directors",
    "Policies",
    "BoardDocs - Meeting Agendas",
    "Board of Education Policies ",
    'Board Agendas',
    "NEOLA Board Policy",
    'NEOLA Board of Education Policies',
    'Neola Board Policies', 
    'NEOLA Board of Education Policy Manual',
    'NEOLA Policies', 
    'Meetings, Agendas, Information',
    'School Board Policies and Guidelines',
    "School Board Policies and Bylaws",
    "NEOLA School Board Policies",
    "Board of School Trustees Policy Manual",
    "School Board Policy	",
    "School Board Policies & Administrative Regulations	",
    "School Board Policy",
    "Board of School Trustees",
    "SCHOOL POLICIES AND GUIDELINES"
]
df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None
# let's check what's left
df["title_2"].value_counts().index[:20]

Index(['Arlington Public Schools', 'Board Policy',
       'Central Valley School District', 'Meade County, South Dakota',
       'Sumter District Schools', 'Santa Clara County Office of Education',
       'Green Local Schools', 'eGovernance System',
       'Monticello Central School District',
       'Community High School District 99', 'Putnam County School District',
       '8485 Homestead, Zeeland, MI 49464 Phone: 616-748-5637',
       'Iowa City Community School District',
       '2045 School Street North Collins, NY 14111', 'Barry ISD',
       'Adena Local Schools',
       '801 Corporate Centre Drive | O'Fallon, MO 63368 | Phone: 636-851-4000',
       '1323 E. 7th Street, Lockport, IL 60441',
       'Phone: 330-627-2181, - Fax: 330-627-2182',
       'Johnson County School District #1'],
      dtype='object', name='title_2')

In [1635]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

89

In [1636]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
3241,https://go.boarddocs.com/wi/threelakessd/Board.nsf/Public,"School Board Agendas, Policies, Rules and Exhibits","6930 West School Street - Three Lakes, WI 54562",https://www.threelakessd.k12.wi.us/,False,True,"6930 West School Street - Three Lakes, WI 54562",True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True
1144,https://go.boarddocs.com/pa/free/Board.nsf/Public,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",Freedom Area School District,http://www.freedomareaschools.org,True,False,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",True,True
1348,https://go.boarddocs.com/oh/barber/Board.nsf/Public,Barberton City School District | 633 Brady Ave Barberton OH 44203 | Phone: 330-753-1025 | FAX: 330-848-0884,Barberton City School District,http://www.barbertonschools.org/,True,False,Barberton City School District | 633 Brady Ave Barberton OH 44203 | Phone: 330-753-1025 | FAX: 330-848-0884,True,True
2484,https://go.boarddocs.com/pa/pmsd/Board.nsf/Public,Pocono Mountain School District,"135 Pocono Mountain School Road, PO Box 200, Swiftwater, PA 18370",http://www.pmsd.org,False,True,"135 Pocono Mountain School Road, PO Box 200, Swiftwater, PA 18370",True,True


In [1637]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_1_list = [
    "School Board Agendas, Minutes, and Policies",
    "School Board Policies, Bylaws, and Guidelines",
    "School Policies and Guidelines",
    "School Board Policy Manual",
    "Board of School Trustees Policy",
    "School Board Agendas and Policies",
    "SchoolBoard Policy Manual",
    "School Board Agendas, Policies, Rules and Exhibits",
    "School Board Policy",
    "SchoolBoard Policy Manual ",
    "School District Policies ",
    "Board Meetings and Policies",
    "Board Management System",
    "Board of Education Policy and Guidelines",
    "Board Policy and By-Laws",
    "Policy and Bylaws",
    "Board Bylaws & Policies",
    "Bylaws & Policies",
    "Board of Trustees Policies",
    "Policies - Bylaws",
    "Bylaws and Policies",
    "NEOLA Policies",
    "Board of Education Bylaws and Policies",
    "School Board Policy Manual",
    "Board Administrative Guidelines , Bylaws, Forms and Policies",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board Policies, Bylaws, Administrative Guidelines, Forms",
    "Board Agendas and Policies",
    "Policies of the Board of Education",
    "School Board Policies and Guidelines",
    "Corporation Board Policies and Guidelines",
    "NEOLA Board Policies and By-Laws",
    "Policies & Bylaws",
    "Board of Education Policies and Guidelines",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy Handbook",
    "Board of Education Policies",
    "Board Bylaws and Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Board Policies and Administrative Guidelines",
    "Board of Education NEOLA Policy",
    "Board of Education Policies and Administrative Guidelines",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "Board Policy",
    "Policies and Administrative Guidelines",
    "Board of Education Policy and Administrative Guidelines",
    "Policies & Administrative Guidelines",
    "Meetings, Agendas, Policy Manual",
    "Joint Operating Committee (JOC) Policy Manual",
    "Board Policy & Guidelines",
    "BOARD OF TRUSTEES",
    "School Board Policies, Meeting Agendas and Minutes",
    "Board Policies and By-laws",
    "Board of Regents",
    "Board of Education Policies and By-Laws",
    "Board of Education Bylaws and Policies/Administrative Guidelines",
    "BoardDocs LT",
    "Fairless District Policy Manual",
    "Board of Education Bylaws & Policies",
    "Board Policies, Administrative Guidelines and Forms",
    "Board Policy & Bylaws",
    "Success for all in the 21st Century . . ."
]

df.loc[df["title_1"].isin(remove_title_1_list), "title_1"] = None

In [1638]:
# I asked ChatGPT to inspect and find more titles I can remove

remove_title_2_list = [
    "POLICY",
    "Neola Board Policies & Guidelines",
    "Board of Education Policy and Guidelines",
    "NEOLA Policy Manual",
    "Board Policy",
    "Board Policies and Guidelines",
    "Policies And Administrative Guidelines",
    "Board Policies, Administrative Guidelines, and Forms",
    "Board of Education Policy",
    "Board Bylaws and Policies",
    "Policies - Bylaws",
    "Board of Education Bylaws and Policies",
    "Neola Board Policy & Administrative Guidelines",
    "Board Policy Handbook",
    "Policies of the Board of Education",
    "NEOLA Board Policy Manual",
    "Policies & Administrative Guidelines",
    "Policies & Bylaws",
    "Board Bylaws and Policies",
    "Board Policies",
    "Board of Education Policies",
    "Board of Education Policy and Administrative Guidelines",
    "Board Policy Manual",
    "Board of Education Policies and Administrative Guidelines",
    "BoardDocs LT",
    "Policies - Bylaws",
    "Policies of the Board of Education",
    "Board Policy and By-Laws",
    "Policies and Administrative Guidelines",
    "Board of Education Bylaws & Policies",
    "Board Policy",
    "Policies & Administrative Guidelines",
    "Board of Education School Policies",
    "School Board",
    "School Board Policies & Administrative Regulations",
    "School Board Policy",
    "Board of School Trustees Policy Manual",
    "School Board Policies and Guidelines",
    "School Board Policy ",
    "Board Policy",
    "Bylaws and Policies",
    "Meetings, Agenda and Information",
    "eGovernance System",
    "Meeting Packets",
    "Opportunity. Equity. Social Justice.",
    "Neola Board Policies & Guidelines",
    "Board Of Education Policy",
    "Board of Education NEOLA Policy",
    "Board Of Education Policies",
    "NEOLA - Board of Education Policies",
    "Board Policy Manual and Administrative Guidelines",
    "Policies & By-Laws",
    "Providing today's students opportunities to become tomorrow's leaders",
    "Board of School Trustees Policy",
    "Corporation Policies",
    "Board of Education Neola Polcies",
    "NEOLA Board Policies and By-Laws",
    "Board of Education Policy /Administrative Guidelines",
    "Board of Education Policies and Bylaws",
    "Board of Education Policies and Guidelines",
    "Neola Board of Education Policy Manual",
    "Neola - Board Policy",
    "Neola Board Policy",
    "Creating the Greatest Opportunities for Our Students"
]

df.loc[df["title_2"].isin(remove_title_2_list), "title_2"] = None

In [1639]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

79

In [1640]:
# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
538,https://go.boarddocs.com/ny/trivc/Board.nsf/Public,Tri-Valley Central School District Board of Education,Tri-Valley Central School District,http://www.trivalleycsd.org/,False,False,None,True,True
3326,https://go.boarddocs.com/mi/trav/Board.nsf/Public,TCAPS School Board Policies & Guidelines,Traverse City Area Public Schools,http://www.tcaps.net/board,False,False,None,True,True
1144,https://go.boarddocs.com/pa/free/Board.nsf/Public,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",Freedom Area School District,http://www.freedomareaschools.org,True,False,"1702 School Street | Freedom, PA 15042 | (724) 775-5464",True,True
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,False,False,None,True,True
3440,https://go.boarddocs.com/ny/secsd/Board.nsf/Public,Sherburne-Earlville Central School District,"15 School Street, Sherburne, NY 13460 | P: 607.674.7300",https://www.secsd.org/,False,True,"15 School Street, Sherburne, NY 13460 | P: 607.674.7300",True,True


In [1641]:
# since a lot of it is from addresses,
# I will remove those where we have ported to the address field

df.loc[df["title_1"] == df["address"], "title_1"] = None
df.loc[df["title_2"] == df["address"], "title_2"] = None

In [1642]:
# now, we check for the intersections again

# let's check if they have the word school

df["title_1_has_school"] = df["title_1"].str.contains("school",case=False,na=False)
df["title_2_has_school"] = df["title_2"].str.contains("school",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school"] & df["title_2_has_school"]].shape[0]

20

In [1643]:
# from 97 to 38, pretty good

# check out these rows with both cols having schools
df[df["title_1_has_school"] & df["title_2_has_school"]].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,False,False,None,True,True
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,False,False,None,True,True
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County Schools,Jefferson County School Board,https://www.jeffersonschools.net,False,False,None,True,True
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,False,False,None,True,True
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,Laurel School Board Agendas and Policy Manual,Laurel School District,http://www.laurel.k12.pa.us,False,False,None,True,True


In [1644]:
# now, let's check those with both titles still intact

n = df[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(f"Number of rows with both cols intact: {n}")

Number of rows with both cols intact: 156


In [1645]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

4

In [1646]:
# check out these with both
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]]

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district
538,https://go.boarddocs.com/ny/trivc/Board.nsf/Public,Tri-Valley Central School District Board of Education,Tri-Valley Central School District,http://www.trivalleycsd.org/,False,False,None,True,True,True,True
769,https://go.boarddocs.com/pa/epns/Board.nsf/Public,EAST PENNSBORO AREA SCHOOL DISTRICT,East Pennsboro Area School District,http://www.epasd.org,False,False,None,True,True,True,True
1593,https://go.boarddocs.com/oh/labr/Board.nsf/Public,Policies of the LaBrae Local School District,LaBrae Local School District,https://labrae.school,False,False,None,True,True,True,True
2031,https://go.boarddocs.com/wi/vasd/Board.nsf/Public,Verona Area School District,Verona Area School District,http://www.verona.k12.wi.us,False,False,None,True,True,True,True


In [1647]:
# we can remove all in title_1
df.loc[df["title_1_has_school_district"] & df["title_2_has_school_district"], "title_1"] = None

In [1648]:
# let's make a school district column
# only do those to rows where EXACTLY one column has the word school district

df["title_1_has_school_district"] = df["title_1"].str.contains("school district",case=False,na=False)
df["title_2_has_school_district"] = df["title_2"].str.contains("school district",case=False,na=False)

# number of rows with schools in both cols
df[df["title_1_has_school_district"] & df["title_2_has_school_district"]].shape[0]

0

In [1649]:
# now we can assign the school_district column

df.loc[df["title_1_has_school_district"],"school_district"] = df["title_1"]
df.loc[df["title_1_has_school_district"],"title_1"] = None

df.loc[df["title_2_has_school_district"],"school_district"] = df["title_2"]
df.loc[df["title_2_has_school_district"],"title_2"] = None

In [1650]:
# now let's look at the remaining columns
# first check those where both cols are still intact

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [1651]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,Title1_has_zipcode,Title2_has_zipcode,address,title_1_has_school,title_2_has_school,title_1_has_school_district,title_2_has_school_district,school_district
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,False,False,None,True,False,False,False,NaN
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,False,False,None,False,False,False,False,NaN
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,False,False,None,True,False,False,False,NaN
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,False,False,None,True,True,False,False,NaN
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,False,False,None,True,False,False,False,NaN


In [1652]:
# now, let's check those with policy or policies in their titles

df["title_1_has_policy"] = df["title_1"].str.contains("policy",case=False,na=False) | df["title_1"].str.contains("policies",case=False,na=False)
df["title_2_has_policy"] = df["title_2"].str.contains("policy",case=False,na=False) | df["title_2"].str.contains("policies",case=False,na=False)

# number of rows with policy in any cols
df[df["title_1_has_policy"] | df["title_2_has_policy"]].shape[0]

50

In [1653]:
# declutter columns
df = df.loc[:,["URL","title_1","title_2","home_website","address","school_district"]]

In [1654]:
df.head()

,URL,title_1,title_2,home_website,address,school_district
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,None,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN
1,https://go.boarddocs.com/pa/cali/Board.nsf/Public,None,NaN,www.calsd.org,None,NaN
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,None,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN
3,https://go.boarddocs.com/oh/rlsd/Board.nsf/Public,None,None,https://www.riversidelocalschools.com/,"585 Riverside Drive | Painesville, Ohio 44077 | 440.352.0668 | f 440.639.1959",Riverside Local School District
4,https://go.boarddocs.com/pa/shun/Board.nsf/Public,None,None,http://www.shcsd.org,None,Southern Huntingdon County School District


In [1655]:
# let's try to common title_1 and title_2
# first standardize None and NaN

print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_1"].isna())

1    None
Name: title_1, dtype: object
1    True
Name: title_1, dtype: bool


In [1656]:
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"])
print(df.loc[df["URL"]=="https://go.boarddocs.com/pa/cali/Board.nsf/Public", "title_2"].isna())

1    NaN
Name: title_2, dtype: object
1    True
Name: title_2, dtype: bool


In [1657]:
# set them to None
df.loc[df["title_1"].isna(), "title_1"] = None
df.loc[df["title_2"].isna(), "title_2"] = None

In [1658]:
# check those with both not None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]

70

In [1659]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].sample(5)

,URL,title_1,title_2,home_website,address,school_district
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN
2108,https://go.boarddocs.com/oh/maplecc/Board.nsf/Public,SCHOOL POLICIES AND GUIDELINES,MAPLEWOOD CAREER CENTER,http://www.mwood.cc/,None,NaN
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN


In [1660]:
# get those with phone numbers out
import re

# Function to check if a string contains a phone number
def contains_phone_number(value):
    # Regular expression for phone numbers
    phone_pattern = re.compile(r'\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
    if pd.isnull(value):
        return False
    return bool(phone_pattern.search(str(value)))

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df['contains_phone_number_title_2'] = df['title_2'].apply(contains_phone_number)

In [1661]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
1112,https://go.boarddocs.com/la/pcpsb/Board.nsf/Public,"337 Napoleon Street ● New Roads, Louisiana ● p 225-638-8674 ● f 225-638-3237",Pointe Coupee Parish School System,http://www.pcpsb.net/,None,NaN,True,False
1576,https://go.boarddocs.com/vsba/fairfax/Board.nsf/Public,"8115 Gatehouse Road, Suite 5400 | Falls Church, VA | 571-423-1075",Fairfax County School Board,http://www.fcps.edu,None,NaN,True,False
1987,https://go.boarddocs.com/oh/nls/Board.nsf/Public,600 Lemoyne Rd | Northwood OH | P: (419) 691-3888 | F: (419) 697-2470,Northwood Local Schools,http://www.northwoodschools.org/site/default.aspx?PageID=1,None,NaN,True,False
2150,https://go.boarddocs.com/oh/polaris/Board.nsf/Public,"7285 Old Oak Blvd., | Middleburg Heights, OH | 440-891-7600",Polaris Career Center,http://www.polaris.edu/,None,NaN,True,False
3356,https://go.boarddocs.com/wa/cowa/Board.nsf/Public,City Council Chambers ~ 500 E. Main Street ~ 509-488-5686 ~ www.othellowa.gov,Othello Washington ~ City Council ~ Serving The Community,http://www.othellowa.gov,None,NaN,True,False


In [1662]:
# all the above are address
move_these = both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_1"]]["URL"]
df.loc[df["URL"].isin(move_these), "address"] = df[df["URL"].isin(move_these)]["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [1663]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1608,https://go.boarddocs.com/oh/sclsd/Board.nsf/Public,South Central Local Schools,3305 Greenwich Angling Rd | 419-752-3815,http://www.south-central.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True


In [1664]:
# move those two that are address
move_these = ["https://go.boarddocs.com/oh/sclsd/Board.nsf/Public", "https://go.boarddocs.com/oh/sidn/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_2"]
df.loc[df["URL"].isin(move_these), "title_2"] = None

In [1665]:
both_col_non_na_df = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]
both_col_non_na_df[both_col_non_na_df["contains_phone_number_title_2"]]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2
159,https://go.boarddocs.com/in/resc/Board.nsf/Public,Randolph Eastern School Corporation,7659644994,http://www.resc.k12.in.us,None,NaN,False,True
440,https://go.boarddocs.com/mi/mionline/Board.nsf/Public,Michigan Online School,Phone: (269) 216-6972,https://www.michiganonlineschool.com/,None,NaN,False,True
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,http://www.triton.k12.in.us,None,NaN,False,True
715,https://go.boarddocs.com/mi/brand/Board.nsf/Public,Brandywine Community Schools,269-684-7150,http://www.brandywinebobcats.org,None,NaN,False,True
1611,https://go.boarddocs.com/mi/whitec/Board.nsf/Public,White Cloud Public Schools,231-689-6591,www.whitecloud.net,None,NaN,False,True
1997,https://go.boarddocs.com/mi/chip/Board.nsf/Public,Chippewa Valley Schools,586-723-2004,http://www.cvs.k12.mi.us,None,NaN,False,True
2306,https://go.boarddocs.com/mi/clark/Board.nsf/Public,Clarkston Community Schools,248-623-5400,http://www.clarkston.k12.mi.us,None,NaN,False,True
2383,https://go.boarddocs.com/mi/white/Board.nsf/Public,Whitefish Township Community Schools,(906) 492-3353,http://whitefish.eupschools.org,None,NaN,False,True
2621,https://go.boarddocs.com/in/sgib/Board.nsf/Public,South Gibson School Corporation,812-753-4230,http://www.sgibson.k12.in.us,None,NaN,False,True
2683,https://go.boarddocs.com/mi/slps/Board.nsf/Public,Spring Lake Public Schools,616-846-5500,http://www.springlakeschools.org,None,NaN,False,True


In [1666]:
# move the others to phone numbers
df["phone"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "phone"] = df["title_2"]
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["contains_phone_number_title_2"], "title_2"] = None

In [1667]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None


In [1668]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"School Board Policy Manual 6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [1669]:
# edit the outlier
df.loc[df["URL"]=="https://go.boarddocs.com/pa/daup/Board.nsf/Public", "title_1"] = "6001 Locust Lane, Harrisburg, PA"

In [1670]:
# check those with numbers
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
2878,https://go.boarddocs.com/pa/daup/Board.nsf/Public,"6001 Locust Lane, Harrisburg, PA",Dauphin County Technical School,http://www.dcts.org,None,NaN,False,False,None
3260,https://go.boarddocs.com/wi/chilsd/Board.nsf/Public,530 Main Street,Chilton Public Schools,www.chilton.k12.wi.us,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [1671]:
# move these two that are addresses
move_these = ["https://go.boarddocs.com/pa/daup/Board.nsf/Public", "https://go.boarddocs.com/wi/chilsd/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "address"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
1386,https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public,LSR-7,Learning for Life,http://www.lsr7.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [1672]:
# move the second one to district and delete the two others
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/mo/lsr7sd/Board.nsf/Public", "title_1"] = None

# check
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29 & Schuylkill Technology Center Boards of Directors,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
3456,https://go.boarddocs.com/vsba/vhsl/Board.nsf/Public,Serving Youth Since 1913,Virginia High School League,http://www.vhsl.org/about_vhsl/executive_committee,None,NaN,False,False,None


In [1673]:
df.loc[idx & df["title_1"].str.contains(number_pattern, na=False), "title_1"] = None

In [1674]:
# do the same for title_2
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
42,https://go.boarddocs.com/oh/rid/Board.nsf/Public,Ridgemont Local Schools,"560 W. Taylor Street Mount Victory, OH",http://www.ridgemont.k12.oh.us,None,NaN,False,False,None
356,https://go.boarddocs.com/ca/mendocino/Board.nsf/Public,Mendocino-Lake Community College District,"1000 Hensley Creek Road, Ukiah, CA",https://www.mendocino.edu,None,NaN,False,False,None
1203,https://go.boarddocs.com/in/brownsburg/Board.nsf/Public,Brownsburg Community School Corporation,"310 Stadium Drive Brownsburg, IN",https://www.brownsburg.k12.in.us/,None,NaN,False,False,None


In [1675]:
# move them to address
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"address"] = df["title_2"]
df.loc[idx & df["title_2"].str.contains(number_pattern, na=False),"title_2"] = None

In [1676]:
# check the rest
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
df.loc[idx,:]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None


In [1677]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2108,https://go.boarddocs.com/oh/maplecc/Board.nsf/Public,SCHOOL POLICIES AND GUIDELINES,MAPLEWOOD CAREER CENTER,http://www.mwood.cc/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None


In [1678]:
# take out the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/oh/maplecc/Board.nsf/Public", "title_1"] = None

In [1679]:
# check those with schools in one but not in the other
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_1"].str.contains("school",case=False,na=False) & ~df["title_2"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
985,https://go.boarddocs.com/mi/byr/Board.nsf/Public,Byron Area Schools,Home of the Eagles,http://www.byron.k12.mi.us,None,NaN,False,False,None
1448,https://go.boarddocs.com/mi/csps/Board.nsf/Public,Cedar Springs Public Schools,Board of Education Board Policies,http://www.csredhawks.org,None,NaN,False,False,None
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,Home of the Lake Fenton Blue Devils,http://www.lakefentonschools.org,None,NaN,False,False,None
1980,https://go.boarddocs.com/mi/fwlv/Board.nsf/Public,Fowlerville Community Schools,Home of the Gladiators,http://www.fowlervilleschools.org,None,NaN,False,False,None
2047,https://go.boarddocs.com/ok/okcps/Board.nsf/Public,Oklahoma City Public Schools,Ignite Passion. Instill Pride.,https://go.boarddocs.com/ok/okcps/Board.nsf/,None,NaN,False,False,None
2301,https://go.boarddocs.com/vsba/louisa/Board.nsf/Public,Louisa County Public Schools,Learners' Community,http://www.lcps.k12.va.us/education/components/scrapbook/default.php?sectiondetailid=1308&PHPSESSID=00a5e9972c66fd8162a098aed2356931,None,NaN,False,False,None
2819,https://go.boarddocs.com/in/sssc/Board.nsf/Public,Southwest School Corporation,By-Laws and Policies,http://www.swest.k12.in.us,None,NaN,False,False,None
3125,https://go.boarddocs.com/in/valp/Board.nsf/Public,Valparaiso Community Schools,Home of the Vikings,http://www.valpo.k12.in.us,None,NaN,False,False,None
3126,https://go.boarddocs.com/oh/zville/Board.nsf/Public,Zanesville City Schools District,Home of the Blue Devils,http://www.zanesville.k12.oh.us,None,NaN,False,False,None
3164,https://go.boarddocs.com/mi/clio/Board.nsf/Public,Clio Area Schools,Home of the Mustangs,http://www.clioschools.org,None,NaN,False,False,None


In [1680]:
# all slogans in second col, delete
df.loc[idx, "title_2"] = None

In [1681]:
# do it for the second col
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None
3443,https://go.boarddocs.com/mi/tda/Board.nsf/Public,THE DEARBORN ACADEMY,Public Charter School,http://www.thedearbornacademy.org,None,NaN,False,False,None


In [1682]:
# edit the outlier
df.loc[df["URL"] == "https://go.boarddocs.com/mi/tda/Board.nsf/Public", "title_2"] = None
idx = (~df["title_1"].isna()) & (~df["title_2"].isna())
idx &= df["title_2"].str.contains("school",case=False,na=False) & ~df["title_1"].str.contains("school",case=False,na=False)
df.loc[idx]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
415,https://go.boarddocs.com/mabe/calvert/Board.nsf/Public,Calvert County Board of Education,Calvert County Public Schools,http://www.calvertnet.k12.md.us,None,NaN,False,False,None
532,https://go.boarddocs.com/vsba/pwcs/Board.nsf/Public,Launching Thriving Futures,Prince William County Public Schools,https://www.pwcs.edu/,None,NaN,False,False,None
1054,https://go.boarddocs.com/ga/fcss/Board.nsf/Public,Where Students Come First,Fulton County Schools,https://portal.fultonschools.org/Pages/default.aspx,None,NaN,False,False,None
1214,https://go.boarddocs.com/vsba/scs/Board.nsf/Public,Together - We Prepare Our Students for Their Future,Spotsylvania County Public Schools,http://www.spotsylvania.k12.va.us/,None,NaN,False,False,None
3308,https://go.boarddocs.com/in/hses/Board.nsf/Public,For additional information click the HOUSE icon to view streamed/archived board meeting videos.,Hamilton Southeastern Schools,https://www.hseschools.org/meet-hse/board,None,NaN,False,False,None
3393,https://go.boarddocs.com/mo/nixa/Board.nsf/Public,Board of Education Meeting Information,Nixa Public Schools,http://www.nixapublicschools.net,None,NaN,False,False,None
3430,https://go.boarddocs.com/mn/d196/Board.nsf/Public,"Educating, developing, and inspiring our students for lifelong success.",Rosemount - Apple Valley - Eagan Public Schools,http://www.district196.org/,None,NaN,False,False,None


In [1683]:
# delete the first col
df.loc[idx,"title_1"] = None

In [1684]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,Ripley-Ohio-Dearborn Special Education Cooperative,ROD Board Policy,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [1685]:
# for those with the word policy, make it the first col
policy_pattern = r'polic(y|ies)'
idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/52244175.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df["title_2"].str.contains(policy_pattern, case=False,na=False)


In [1686]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None


In [1687]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/2999244493.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False)]


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
283,https://go.boarddocs.com/oh/putesc/Board.nsf/Public,Putnam County Educational Services Center District Policy Manual,Putnam County Educational Service Center,www.putnamcountyesc.org,None,NaN,False,False,None
628,https://go.boarddocs.com/oh/cvcc/Board.nsf/Public,Cuyahoga Valley Career Center Bylaws and Policies,Cuyahoga Valley Career Center,http://www.cvccworks.edu/Default.aspx,None,NaN,False,False,None
1050,https://go.boarddocs.com/in/rodsped/Board.nsf/Public,ROD Board Policy,Ripley-Ohio-Dearborn Special Education Cooperative,http://www.rodspecialeducation.org,None,NaN,False,False,None
1161,https://go.boarddocs.com/mi/jon/Board.nsf/Public,JCS School Board Policies and Guidelines,Jonesville Community Schools,NaN,None,NaN,False,False,None
1292,https://go.boarddocs.com/oh/nwoesc/Board.nsf/Public,"NwOESC Board Agendas, Minutes, By-Laws and Policy Manual",Northwest Ohio Educational Service Center,https://www.nwoesc.org/,None,NaN,False,False,None
1537,https://go.boarddocs.com/oh/moesc/Board.nsf/Public,Mid-Ohio Policies,Mid-Ohio ESC,www.moesc.net,None,NaN,False,False,None
2720,https://go.boarddocs.com/mi/giresa/Board.nsf/Public,Board Policies and Bylaws,Gratiot-Isabella Regional Education Service District,http://www.giresd.net,None,NaN,False,False,None
3326,https://go.boarddocs.com/mi/trav/Board.nsf/Public,TCAPS School Board Policies & Guidelines,Traverse City Area Public Schools,http://www.tcaps.net/board,None,NaN,False,False,None
3407,https://go.boarddocs.com/oh/tructc/Board.nsf/Public,TCTC Board Policies,Trumbull Career and Technical Center,http://www.tctchome.com,None,NaN,False,False,None
3525,https://go.boarddocs.com/pa/vnang/Board.nsf/Public,Venango Technology Center Policy Manual,Venango Technology Center,http://www.vtc1.org,None,NaN,False,False,None


In [1688]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/786511925.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains(policy_pattern,case=False,na=False), "title_1"] = None


,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City Public Schools,Falls Church City School Board,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,Utah Education and Telehealth Network,UETN Governing Board,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County Schools,Jefferson County School Board,https://www.jeffersonschools.net,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [1689]:
# do the same for board
idx = df["title_2"].str.contains("board", case=False,na=False)
df.loc[idx, ["title_1", "title_2"]] = df.loc[idx,["title_2","title_1"]].values

In [1690]:
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False)]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2060,https://go.boarddocs.com/vsba/fccpsva/Board.nsf/Public,Falls Church City School Board,Falls Church City Public Schools,https://www.fccps.org/page/school-board,None,NaN,False,False,None
2889,https://go.boarddocs.com/ut/uen/Board.nsf/Public,UETN Governing Board,Utah Education and Telehealth Network,http://www.uetn.org,None,NaN,False,False,None
2963,https://go.boarddocs.com/fl/jcsd/Board.nsf/Public,Jefferson County School Board,Jefferson County Schools,https://www.jeffersonschools.net,None,NaN,False,False,None
3350,https://go.boarddocs.com/in/evsc/Board.nsf/Public,EVSC School Board Meetings,Evansville Vanderburgh School Corporation,www.evscschools.com,None,NaN,False,False,None


In [1691]:
# can remove title_1 for these
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna()) & df["title_1"].str.contains("board",case=False,na=False), "title_1"] = None
df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1729,https://go.boarddocs.com/co/eepto/Board.nsf/Public,"Wellington, CO",Eyestone Elementary PTO,https://eye.psdschools.org/,None,NaN,False,False,None
2229,https://go.boarddocs.com/ca/sjccd/Board.nsf/Public,College of the Siskiyous,Siskiyou Joint Community College District,http://www.siskiyous.edu/,None,NaN,False,False,None
2478,https://go.boarddocs.com/ca/hawking/Board.nsf/Public,Hawking STEAM Charter School,Hawking STEAM Charter Schools,https://www.hawkingschools.org/,None,NaN,False,False,None
3046,https://go.boarddocs.com/mi/badax/Board.nsf/Public,BAPS,Home of the Hatchets,www.badaxeps.org,None,NaN,False,False,None
3257,https://go.boarddocs.com/in/fcsc/Board.nsf/Public,Franklin Township Community School Corp.,Franklin Township Community School Corporation,http://www.ftcsc.k12.in.us/,None,NaN,False,False,None


In [1692]:
# just do case by case
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/sjccd/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/ca/hawking/Board.nsf/Public", "title_1"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/mi/badax/Board.nsf/Public", "title_2"] = None

df.loc[df["URL"]=="https://go.boarddocs.com/in/fcsc/Board.nsf/Public", "title_1"] = None

df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())]

,URL,title_1,title_2,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [1693]:
# check those with both not None
n = df.loc[(~df["title_1"].isna()) & (~df["title_2"].isna())].shape[0]
print(n)
assert n == 0

0


In [1694]:
# now we can combine both
df.loc[df["title_1"].isna(), "title_1"] = df["title_2"]
df = df.drop("title_2", axis=1)

In [1695]:
# see what is left
df.loc[~df["title_1"].isna()].shape[0]

1819

In [1696]:
df.loc[~df["title_1"].isna()].sample(5)

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2097,https://go.boarddocs.com/tn/wcschools/Board.nsf/Public,Wilson County Schools,https://www.wcschools.com,"415 Harding Drive, Lebanon, TN 37087 | 615-444-3282",NaN,False,False,None
3149,https://go.boarddocs.com/mi/doveacademy/Board.nsf/Public,Dove Academy of Detroit,https://doveacademy.net/,"20001 Wexford St., Detroit, Michigan 48234 Phone: (313) 366-9110",NaN,False,False,None
1954,https://go.boarddocs.com/pa/hcct/Board.nsf/Public,Huntingdon County Career and Technology Center,www.hcctc.org,"11893 Technology Drive, P.O. Box E, Mill Creek, PA 17060",NaN,False,False,None
2351,https://go.boarddocs.com/mi/wmaviation/Board.nsf/Public,West Michigan Aviation Academy,https://www.westmichiganaviation.org/,"5363 44th Street SE, Grand Rapids, MI 49512 Phone: (616) 446-8886",NaN,False,False,None
3779,https://go.boarddocs.com/nj/ocsdnj/Board.nsf/Public,Ocean City Board of Education,https://www.ocsdnj.org/page/board-of-education,"501 Atlantic Avenue, Suite #1, Ocean City, NJ 08226",NaN,False,False,None


In [1697]:
# check those with numbers
df.loc[df["title_1"].str.contains(number_pattern, na=False)].shape[0]

134

In [1698]:
df.loc[df["title_1"].str.contains(number_pattern, na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
38,https://go.boarddocs.com/mo/mcr1/Board.nsf/Public,Macon County R-1 Schools,http://www.macon.k12.mo.us/,"702 North Missouri  Macon, Missouri 63552  (660) 395-6164",NaN,False,False,None
49,https://go.boarddocs.com/ks/susdks/Board.nsf/Public,Seaman USD 345,http://www.seamanschools.org,"901 NW Lyman Road | Topeka, KS 66608 | Ph: (785) 575-8600",NaN,False,False,None
76,https://go.boarddocs.com/pa/iu11/Board.nsf/Public,Tuscarora Intermediate Unit 11,https://www.tiu11.org/,None,NaN,False,False,None
125,https://go.boarddocs.com/wi/pesh/Board.nsf/Public,341 NORTH EMERY AVENUE,www.peshtigo.k12.wi.us,None,PESHTIGO SCHOOL DISTRICT,False,False,None
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
...,...,...,...,...,...,...,...,...
3806,https://go.boarddocs.com/ks/usd315/Board.nsf/Public,Colby Public Schools USD 315,http://www.colbyeagles.org/,600 W 3rd St. | Colby KS 67701-2000 | p 785-460-5000 | f 785-460-5050,NaN,False,False,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,False,True,None
3820,https://go.boarddocs.com/ks/usd311/Board.nsf/Public,Pretty Prairie USD 311 KS,https://www.usd311.com/,"206 E Main, P.O. Box 218 Pretty Prairie, Kansas 67570",NaN,False,False,None
3877,https://go.boarddocs.com/pa/neiu/Board.nsf/Public,Northeastern Educational Intermediate Unit 19,http://www.iu19.org,1200 Line Street Archbald PA 18403,NaN,False,False,None


In [1699]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

38

In [1700]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None


In [1701]:
# move those without alphabets to the phone col

alphabet_pattern = r'[a-zA-Z]'
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)) ]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
680,https://go.boarddocs.com/pa/prtg/Board.nsf/Public,(814) 736-9636,www.portageareasd.org,"84 Mountain Avenue, Portage, PA 15946",NaN,True,True,None
1029,https://go.boarddocs.com/mi/wake/Board.nsf/Public,(906) 224-7211,http://www.wmschools.org,None,Wakefield-Marenisco School District,True,True,None
1765,https://go.boarddocs.com/pa/iu14/Board.nsf/Public,610-987-2248,http://www.berksiu.org/,"1111 Commons Boulevard, PO Box 16050, 19612-6050",NaN,True,True,None
1924,https://go.boarddocs.com/wi/shorewood/Board.nsf/Public,(414) 963-6901,https://www.shorewood.k12.wi.us,"Shorewood School District | 1701 East Capitol Drive | Shorewood, Wisconsin 53211",NaN,True,True,None
1985,https://go.boarddocs.com/pa/roch/Board.nsf/Public,724-775-7500,http://www.rasd.org,"540 Reno Street, Rochester, PA 15074",NaN,True,True,None
2028,https://go.boarddocs.com/mi/fowler/Board.nsf/Public,(989)593-2250,http://www.fowlerschools.net,"700 S Main Street, Fowler MI 48835",NaN,True,True,None
2525,https://go.boarddocs.com/nj/burlingtontwp/Board.nsf/Public,(609) 387-3955,https://burltwpsch.org/,None,Burlington Township School District,True,True,None
2767,https://go.boarddocs.com/in/jcdc/Board.nsf/Public,812-689-4114,http://www.jaccendel.k12.in.us,"723 N Buckeye St, Osgood IN 47037",NaN,True,True,None
2874,https://go.boarddocs.com/wi/hfj1/Board.nsf/Public,(262) 673-3155,www.hjt1.org,"School District of Hartford Jt. #1, 402 W. Sumner St. Hartford, WI 53027",NaN,True,True,None
3159,https://go.boarddocs.com/wi/wawmsd/Board.nsf/Public,414-604-3000,http://www.wawmsd.org,"West Allis-West Milwaukee School District, 9333 W. Lincoln Avenue, West Allis, WI 53227",NaN,True,True,None


In [1702]:
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "phone"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"] & (~df["title_1"].str.contains(alphabet_pattern,na=False)), "title_1"] = None


In [1703]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]].shape[0]

28

In [1704]:
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None


In [1705]:
# the relevant info in the column is either phone, address or both
# the row with the website already has the website col filled
# first get those that are phone only

In [1706]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*[Pp](hone)[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/2831296269.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1574,https://go.boarddocs.com/oh/swissohio/Board.nsf/Public,Phone: 740-472-5801,https://swissohio.k12.oh.us,"304 Mill Street Woodsfield, OH 43793",NaN,True,True,None
2140,https://go.boarddocs.com/wi/solon/Board.nsf/Public,Phone: (715) 378-2263,https://www.solonk12.net,"School District of Solon Springs, 8993 E Baldwin Ave, Solon Springs, WI 54873",NaN,True,True,None
2160,https://go.boarddocs.com/oh/galionoh/Board.nsf/Public,Phone - 419-468-3432,https://www.galionschools.org/,"Galion City Schools - 470 Portland Way North - Galion, OH 44833",NaN,True,True,None
3816,https://go.boarddocs.com/fl/semi/Board.nsf/Public,Phone - 407-320-0000,www.scps.k12.fl.us,"400 E. Lake Mary Boulevard - Sanford, FL - 32773",NaN,True,True,None


In [1707]:
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3918443988.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3918443988.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [1708]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None


In [1709]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/3636070497.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False)]


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,Phone: 715-257-7511 Fax: 715-257-7502,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,True,True,None
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,(920) 533-8381 | Fax (920) 533 -5726,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,True,True,None
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,"Phone: 330-627-2181, - Fax: 330-627-2182",www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,True,True,None
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,(208) 468-4600 Fax: (208) 468-4638,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,True,True,None
3001,https://go.boarddocs.com/fl/highlfl/Board.nsf/Public,PH: 863-471-5555,www.highlands.k12.fl.us,"School Board of Highlands County, 426 School St., Sebring, FL 33870",NaN,True,True,None
3758,https://go.boarddocs.com/fl/brevco/Board.nsf/Public,* MAIN PHONE: (321) 633-1000,https://www.brevardschools.org/,"2700 JUDGE FRAN JAMIESON WAY, VIERA, FL 32940",NaN,True,True,None


In [1710]:
# Define a pattern to match the words "phone" or "fax" only, allowing other non-alphabet characters
phone_fax_pattern = r'^[^a-zA-Z]*([Pp]hone|PH|MAIN PHONE)?[^a-zA-Z]*([Ff]ax)?[^a-zA-Z]*$'

# Filter the rows
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/464474225.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "phone"] = df["title_1"]
/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_81173/464474225.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['contains_phone_number_title_1'] & df['title_1'].str.contains(phone_fax_pattern, case=False, na=False), "title_1"] = None


In [1711]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,True,True,None
2366,https://go.boarddocs.com/oh/naps/Board.nsf/Public,www.napoleonareaschools.org | 419-599-7015,www.napoleon.k12.oh.us,None,Napoleon Area City School District,True,False,None


In [1712]:
# handle special cases
move_these = ["https://go.boarddocs.com/ks/usd230/Board.nsf/Public","https://go.boarddocs.com/oh/naps/Board.nsf/Public"]
df.loc[df["URL"].isin(move_these), "phone"] = df["title_1"]
df.loc[df["URL"].isin(move_these), "title_1"] = None

In [1713]:
# get the phone numbers

# Apply the function to a specific column (e.g., "title_2")
df['contains_phone_number_title_1'] = df['title_1'].apply(contains_phone_number)
df.loc[df["contains_phone_number_title_1"]]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
134,https://go.boarddocs.com/mi/hartl/Board.nsf/Public,Hartland Consolidated Schools | phone: 810.626.2105 | fax: 810.626.2101,http://www.hartlandschools.us,None,NaN,True,False,None
252,https://go.boarddocs.com/wi/colesd/Board.nsf/Public,"347 Business Highway 141 North, Coleman WI | 920-897-4011",www.coleman.k12.wi.us,None,Coleman School District,True,False,None
547,https://go.boarddocs.com/wi/nfdl/Board.nsf/Public,"1115 Thurke Ave - North Fond du Lac, WI - (920) 929-3750",www.nfdlschools.org,None,NaN,True,False,None
767,https://go.boarddocs.com/mi/cmps/Board.nsf/Public,Central Montcalm Public School | Office: 989-831-2001 | Fax: 989-831-2010,http://www.central-montcalm.org,None,NaN,True,False,None
856,https://go.boarddocs.com/mi/elk/Board.nsf/Public,"Elk Rapids Central Office • 308 Meguzee Point Rd • Elk Rapids, Michigan • Phone: (231) 264-8692 Fax: (231) 264-6538",www.erschools.com,None,NaN,True,False,None
1064,https://go.boarddocs.com/wi/spartan/Board.nsf/Public,"900 E. Montgomery St. Sparta, Wisconsin | (608) 366-3400",https://www.spartan.org,None,Sparta Area School District,True,True,None
1445,https://go.boarddocs.com/oh/hunt/Board.nsf/Public,"188 Huntsman Road | Chillicothe, OH | P: 740.663.5892 | F: 740.663.6078",http://www.huntsmen.org/,None,Huntington Local School District,True,False,None
2128,https://go.boarddocs.com/ut/nebo/Board.nsf/Public,"350 S. Main | Spanish Fork, Utah | 801-354-7400",http://www.nebo.edu,None,Nebo School District Board of Education,True,False,None
2495,https://go.boarddocs.com/in/brem/Board.nsf/Public,Bremen Public Schools | Phone: (574) 546-3929 | Fax: (574) 546-6303 | School Board Policies and Guidelines,https://www.bps.k12.in.us,None,NaN,True,False,None
2808,https://go.boarddocs.com/wi/ashland/Board.nsf/Public,"District Office - 2000 Beaser Avenue; Ashland, WI; (715) 682-7080",http://www.ashland.k12.wi.us/,None,School District of Ashland Board of Education,True,False,None


In [1714]:
# these are all addresses, move them
df.loc[df["contains_phone_number_title_1"], "address"] = df["title_1"]
df.loc[df["contains_phone_number_title_1"], "title_1"] = None

In [1715]:
# now check the remaining ones
print(df[~df["title_1"].isna()].shape[0])
df[~df["title_1"].isna()].sample(5)

1781


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3727,https://go.boarddocs.com/pa/cccc/Board.nsf/Public,Clarion County Career Center,http://www.clarioncte.org,None,NaN,False,False,None
2477,https://go.boarddocs.com/vsba/glou/Board.nsf/Public,Gloucester County Public Schools,http://gets.gc.k12.va.us/,"6099 T.C. Walker Road | Gloucester, VA 23061 | 804-693-5300",NaN,False,False,None
3793,https://go.boarddocs.com/sc/acs/Board.nsf/Public,"Together We Can Do This-Rebuild, Redirect, Redefine",http://www.acs.k12.sc.us/,"3249 Allendale Fairfax Hwy Fairfax, SC 29827 803-584-4603",NaN,False,False,None
2763,https://go.boarddocs.com/vsba/wythe/Board.nsf/Public,Wythe County Public Schools,http://wythe.k12.va.us/,"1570 West Reservoir Street | Wytheville, VA 24382 | Ph: 276-228-5411 | Fx: 276-228-9192",NaN,False,False,None
2368,https://go.boarddocs.com/ks/usd435/Board.nsf/Public,Abilene USD 435,http://www.abileneschools.org/USD435/Main.html,"P.O. Box 639, 213 N Broadway | Abilene, KS 67410 | Ph: 785-263-2630 | Fx: 785-263-7610",NaN,False,False,None


In [1716]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None


In [1717]:
# the agenda value keep coming up
# let's check for all rows on this
df.loc[df["title_1"].str.contains("agenda",case=False,na=False)]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
157,https://go.boarddocs.com/il/sd163/Board.nsf/Public,Board Meeting Agendas,http://www.sd163.com,None,NaN,False,False,None
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,Laurel School Board Agendas and Policy Manual,http://www.laurel.k12.pa.us,None,Laurel School District,False,False,None
901,https://go.boarddocs.com/nd/bsd7/Board.nsf/Public,"Agendas, Minutes, District Information, Policies and Events",http://www.belcourt.k12.nd.us/education/components/scrapbook/default.php?sectiondetailid=1376&,None,Turtle Mountain Community Schools/Belcourt School District #7,False,False,None
1163,https://go.boarddocs.com/ca/vibrantminds/Board.nsf/Public,"Board Agendas, Minutes, and Policies",https://www.vibrantminds.us/,"412 W. CARL KARCHER WAY ANAHEIM, CA 92801 | ​714-563-2390",NaN,False,False,None
1736,https://go.boarddocs.com/pa/uncf/Board.nsf/Public,"Meetings, Agendas, and Information",http://www.ucfsd.org,None,Unionville-Chadds Ford School District,False,False,None
1800,https://go.boarddocs.com/pa/boyr/Board.nsf/Public,"Meetings, Agendas, Information",http://www.boyertownasd.org,None,NaN,False,False,None
2077,https://go.boarddocs.com/mo/unionrxi/Board.nsf/Public,"Meetings, Agendas, and Information",https://www.unionrxi.org,None,Union R-XI School District,False,False,None
2472,https://go.boarddocs.com/pa/iu24/Board.nsf/Public,Board Agendas and Policy Manual,http://www.cciu.org,"455 Boot Road, Downingtown, PA 19335 | 484-237-5000",NaN,False,False,None
2521,https://go.boarddocs.com/oh/brightoh/Board.nsf/Public,"Meeting Agendas, Minutes and Policies",https://www.blsd.us,None,Bright Local School District,False,False,None
2646,https://go.boarddocs.com/oh/mayoh/Board.nsf/Public,Meeting Agendas & Minutes,http://www.mayfieldschools.org,None,Mayfield City School District,False,False,None


In [1718]:
# remove them

df.loc[df["title_1"].str.contains("agenda",case=False,na=False), "title_1"] = None

In [1719]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [1720]:
# get those with numbers first

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
723,https://go.boarddocs.com/il/thsd211/Board.nsf/Public,"G.A. McElroy Administration Center, 1750 South Roselle Road, Palatine, Illinois",http://www.adc.d211.org,None,NaN,False,False,None
882,https://go.boarddocs.com/il/asd4/Board.nsf/Public,"222 N. Kennedy Drive, Addison, IL",http://www.asd4.org,None,Addison School District 4,False,False,None
977,https://go.boarddocs.com/mo/rockport/Board.nsf/Public,"600 S Nebraska St | Rock Port, MO",https://rockport.k12.mo.us/,None,Rock Port R-II School District,False,False,None
2134,https://go.boarddocs.com/in/pike/Board.nsf/Public,"Administrative Services Center - 6901 Zionsville Road, Indianapolis, IN & via Live Streaming at: https://www.youtube.com/channel/UCsnM2UOzNfDocPPLZzktSOg",http://www.pike.k12.in.us/,None,Metropolitan School District of Pike Township,False,False,None
2627,https://go.boarddocs.com/nj/hhboe/Board.nsf/Public,"316-A Seventh Ave Haddon Heights, New Jersey",http://gogarnets.com/,None,Haddon Heights School District,False,False,None
2729,https://go.boarddocs.com/ca/cvesd/Board.nsf/Public,"84 East J Street , Chula Vista , CA91910",https://www.cvesd.org/,None,Chula Vista Elementary School District,False,False,None
3287,https://go.boarddocs.com/wi/afasd/Board.nsf/Public,"201 W. 6th Street, Friendship, WI",https://www.afasd.net/,None,ADAMS-FRIENDSHIP AREA SCHOOL DISTRICT,False,False,None


In [1721]:
# move them to the address
idx = df["title_1"].str.contains(",",na=False) & df["address"].isna() & df["title_1"].str.contains(number_pattern)
df.loc[idx,"address"] = df["title_1"]
df.loc[idx,"title_1"] = None

In [1722]:
# check those with commas
# could be addresses
# check those without addresses
df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),:]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2505,https://go.boarddocs.com/wv/brooke/Board.nsf/Public,"Brooke County Schools - Excellence, Tradition and Bruin Pride!",https://www.brooke.k12.wv.us,None,NaN,False,False,None
2710,https://go.boarddocs.com/ak/swrsdak/Board.nsf/Public,"... educating our future, guided by our past",http://www.swrsd.org,None,Southwest Region School District,False,False,None
2824,https://go.boarddocs.com/mo/wpr7sd/Board.nsf/Public,"Excellence in Education, Service, Life.",https://www.zizzers.org,None,West Plains School District,False,False,None


In [1723]:
# move the first one to address and remove the rest, which are slogans
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/co/eepto/Board.nsf/Public", "title_1"] = None

df.loc[df["title_1"].str.contains(",",na=False) & df["address"].isna(),"title_1"] = None

In [1724]:
# check the remaining ones
print(df.loc[~df["title_1"].isna()].shape[0])
df.loc[~df["title_1"].isna()].sample(5)

1757


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1569,https://go.boarddocs.com/in/crawfordsville/Board.nsf/Public,Crawfordsville Community School Corporation,https://www.cville.k12.in.us/,"1000 Fairview Ave. | Crawfordsville, IN 47933 | (765) 362-2342",NaN,False,False,None
830,https://go.boarddocs.com/tn/brist/Board.nsf/Public,Bristol Tennessee City Schools,http://www.btcs.org,"615 Martin Luther King Jr. Blvd. | Bristol, Tennessee 37620 | (423) 652-9451",NaN,False,False,None
733,https://go.boarddocs.com/az/asdb/Board.nsf/Public,Arizona State Schools for the Deaf and the Blind,https://asdb.az.gov/,"1200 W. Speedway Blvd. | Tucson, AZ 85745 | p 520.770.3362 | f 520.770.3711",NaN,False,False,None
598,https://go.boarddocs.com/il/triad/Board.nsf/Public,Triad Community Schools - Unit District #2,http://www.triadunit2.org,"203 East Throp Street | Troy, Illinois 62294 | 618.667.5400",NaN,False,False,None
3404,https://go.boarddocs.com/in/stoh/Board.nsf/Public,School Town of Highland,http://www.highland.k12.in.us,None,NaN,False,False,None


In [1725]:
# check those with address na too
print(df.loc[~df["title_1"].isna() & df["address"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["address"].isna()].sample(5)

559


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3614,https://go.boarddocs.com/mi/nwlo/Board.nsf/Public,New Lothrop Area Public Schools,http://www.newlothrop.k12.mi.us,None,NaN,False,False,None
1080,https://go.boarddocs.com/mi/pent/Board.nsf/Public,Pentwater Public Schools,http://www.pentwaterschools.net,None,NaN,False,False,None
735,https://go.boarddocs.com/pa/iu29/Board.nsf/Public,Schuylkill Intermediate Unit 29,http://www.iu29.org,None,NaN,False,False,None
3748,https://go.boarddocs.com/mi/nile/Board.nsf/Public,Niles Community Schools,http://www.nilesschools.org,None,NaN,False,False,None
1645,https://go.boarddocs.com/mi/beld/Board.nsf/Public,Belding Area Schools,http://www.bas-k12.org/,None,NaN,False,False,None


In [1726]:
# check those with school_district na too
print(df.loc[~df["title_1"].isna() & df["school_district"].isna()].shape[0])
df.loc[~df["title_1"].isna() & df["school_district"].isna()].sample(5)

1704


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
1424,https://go.boarddocs.com/ca/rsccd/Board.nsf/Public,Rancho Santiago Community College District,https://www.rsccd.edu,None,NaN,False,False,None
3279,https://go.boarddocs.com/mi/nvs/Board.nsf/Public,Norway-Vulcan Area Schools,http://www.nvknights.org,None,NaN,False,False,None
2683,https://go.boarddocs.com/mi/slps/Board.nsf/Public,Spring Lake Public Schools,http://www.springlakeschools.org,None,NaN,False,True,616-846-5500
1548,https://go.boarddocs.com/mi/lfen/Board.nsf/Public,Lake Fenton Community Schools,http://www.lakefentonschools.org,None,NaN,False,False,None
69,https://go.boarddocs.com/pa/cvsdpa/Board.nsf/Public,Chartiers Valley Board of School Directors,https://www.cvsd.net/about-cvsd/board-of-directors,None,NaN,False,False,None


In [1727]:
# check those with non-alphabets in title_1
non_alpha_pattern = r'[^A-Za-z ]'
print(df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].shape[0])
df.loc[df["title_1"].str.contains(non_alpha_pattern, na=False)].sample(5)

319


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2467,https://go.boarddocs.com/nj/spfk/Board.nsf/Public,Scotch Plains-Fanwood Board of Education,https://www.spfk12.org/,"512 Cedar Street, Scotch Plains, NJ 07076",NaN,False,False,None
1077,https://go.boarddocs.com/oh/napls/Board.nsf/Public,New Albany-Plain Local Schools,https://www.napls.us/our-district/board-of-education,"55 North High Street | New Albany, OH 43054 | 614.855.2040",NaN,False,False,None
467,https://go.boarddocs.com/in/nlawr/Board.nsf/Public,North Lawrence Comm. School Corp.,http://www.nlcs.k12.in.us,"460 W Street, Bedford, IN 47421, Phone: (812)-279-3521",NaN,False,False,None
3679,https://go.boarddocs.com/pa/iu08/Board.nsf/Public,Appalachia Intermediate Unit 8,http://www.iu08.org,None,NaN,False,False,None
3442,https://go.boarddocs.com/ks/usd309/Board.nsf/Public,USD 309 Nickerson - South Hutchinson,http://www.usd309ks.org,"4501 West Fourth | Hutchinson, KS 67501 | Ph: (620) 663-7141 | Fx: 620-663-7148 (Fax)",NaN,False,False,None


In [1728]:
# for those that have addresses set, we can move them to school_district
df.loc[~df["address"].isna() & ~df["title_1"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
2,https://go.boarddocs.com/oh/mapleheights/Board.nsf/Public,Maple Heights City Schools,http://www.mapleschools.com,"5740 Lawn Avenue | Maple Heights, OH 44137 | 216-587-6100",NaN,False,False,None
10,https://go.boarddocs.com/in/centergrove/Board.nsf/Public,Center Grove Community School Corporation,https://www.centergrove.k12.in.us/,"4800 West Stones Crossing Road | Greenwood, IN 46143 | (317) 881-9326",NaN,False,False,None
12,https://go.boarddocs.com/oh/pcc/Board.nsf/Public,Penta Career Center,http://www.pentacareercenter.org/,"9301 Buck Rd. | Perrysburg, Ohio 43551 | High School: 419-666-1120 Adult Education: 419-661-6555",NaN,False,False,None
15,https://go.boarddocs.com/oh/westholmes/Board.nsf/Public,West Holmes Local Schools,https://westholmes.org/,"28 W Jackson St. | Millersburg, OH 44654 | 330-674-3546",NaN,False,False,None
16,https://go.boarddocs.com/mabe/carps/Board.nsf/Public,Caroline County Public Schools,carolineschools.org,"Address: 204 Franklin Street | Denton, MD 21629 | Phone: (410) 479-1460",NaN,False,False,None
...,...,...,...,...,...,...,...,...
3897,https://go.boarddocs.com/co/jeffco/Board.nsf/Public,Jeffco Public Schools Board of Education,http://www.jeffcopublicschools.org/,1829 Denver West Drive | Golden. CO 80401 | (303) 982-6800,NaN,False,False,None
3902,https://go.boarddocs.com/il/cowil/Board.nsf/Public,City of Waukegan,http://www.waukeganil.gov/,"100 N. Martin Luther King Jr. Ave. | Waukegan, IL 60085",NaN,False,False,None
3903,https://go.boarddocs.com/ca/laccd/Board.nsf/Public,Los Angeles Community College District,http://laccd.edu,"770 Wilshire Boulevard, Los Angeles, CA 90017 | (213) 891-2000",NaN,False,False,None
3904,https://go.boarddocs.com/md/stmarysco/Board.nsf/Public,"St. Mary's County, Maryland",http://www.stmarysmd.com,"41770 Baldridge Street | Leonardtown, MD 20650 | 301-475-4200",NaN,False,False,None


In [1729]:
# check if there are any where all three are set
df.loc[~df["address"].isna() & ~df["title_1"].isna() & ~df["school_district"].isna()]

,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone


In [1730]:
# not really sure if it will be entirely correct
# check for those without address set
print(df.loc[df["address"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & ~df["title_1"].isna()].head()

559


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
0,https://go.boarddocs.com/mi/sjs/Board.nsf/Public,St. Joseph Public Schools,https://www.sjschools.org/,None,NaN,False,False,None
6,https://go.boarddocs.com/pa/fcctc/Board.nsf/Public,Franklin County Career and Technology Center,www.franklinctc.com,None,NaN,False,False,None
9,https://go.boarddocs.com/ca/voc/Board.nsf/Public,Vista Oaks Charter School,https://www.vistaoaks.net,None,NaN,False,False,None
20,https://go.boarddocs.com/mi/man/Board.nsf/Public,Manton Consolidated Schools,http://mantonmi.apptegy.us/o/mcs?mode=edit,None,NaN,False,False,None
21,https://go.boarddocs.com/pa/cdsd/Board.nsf/Public,Quality to the Core,http://www.cdschools.org,None,Central Dauphin School District,False,False,None


In [1731]:
# not really sure if it will be entirely correct
# check for those with both unset
print(df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].shape[0])
df.loc[df["address"].isna() & df["school_district"].isna() & ~df["title_1"].isna()].sample(5)

506


,URL,title_1,home_website,address,school_district,contains_phone_number_title_1,contains_phone_number_title_2,phone
3899,https://go.boarddocs.com/mi/atlanta/Board.nsf/Public,Atlanta Community Schools,https:\\www.atlantaschools.us,None,NaN,False,False,None
2216,https://go.boarddocs.com/vsba/dinwiddie/Board.nsf/Public,DINWIDDIE COUNTY SCHOOL BOARD,http://www.dinwiddie.k12.va.us/,None,NaN,False,False,None
207,https://go.boarddocs.com/in/greens/Board.nsf/Public,Greensburg Community Schools,http://www.greensburg.k12.in.us,None,NaN,False,False,None
129,https://go.boarddocs.com/mi/llhs/Board.nsf/Public,Lake Linden-Hubbell Public Schools,www.lakelinden.k12.mi.us,None,NaN,False,False,None
3190,https://go.boarddocs.com/in/fairfiel/Board.nsf/Public,Fairfield Community Schools,http://www.fairfield.k12.in.us,None,NaN,False,False,None


In [1732]:
# check how many we have left
sum(~df["title_1"].isna())

1757

In [1733]:
# send to GPT to deal with it
df.to_csv("test.csv", columns=["title_1","address","school_district"], index=False)

In [1734]:
# I gave GPT-4o the following prompt
"""
This csv has three columns: title_1, address, and school_district. Many values are empty.

Your goal is to move the value from title_1 to either address, or school_district, or simply discard it. Do not overwrite any values.

For example, for the row

Christopher Newport University,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000",

You should put Christopher Newport University as the school_district, which is third column, that is

,"1 Avenue of the Arts Newport News, VA 23606 | (757) 594-7000","Christopher Newport University"

Another example

Quality to the Core,,Central Dauphin School District

You should just drop the title_1, since it is a slogan

,,Central Dauphin School District

Another example

75 Chenango Ave Clinton NY13323,,Clinton Central School District

You should put this in the address column

,75 Chenango Ave Clinton NY13323,Clinton Central School District

Return a csv
"""

# followed by 

"""
Ok. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"
"""

# it returns the following csv

'\nOk. Now instead of moving, add a fourth column that says "address", "school_district", "None" or "drop"\n'

In [1735]:
gpt_df = pd.read_csv("classified_test_by_gpt.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
2169,USD #252 Southern Lyon County,"100 Commercial St. Suite A | Hartford, KS 66854 | Tel: (620) 392-5519 | Fx: 620-392-5841",NaN,school_district
3163,Williamsburg-James City County Public Schools,"PO Box 8783 | 117 Ironbound Road | Williamsburg, VA 23187-8783",NaN,school_district
505,NaN,"2 Pleasant Avenue | Schaghticoke, NY 12154 | (518) 753-4458",Hoosic Valley Central School District,NaN
1792,Placer County Office of Education,"1400 W. Stanford Ranch Road | Rocklin, CA 95765 | Ph: (530) 889-8020 Fx: (866) 840-2941",NaN,school_district
1902,Home of the Panthers!,"200 N Farm Road | Oconto Falls, WI 54154 | Phone (920) 848-4471",NaN,school_district
1981,Jackson Preparatory and Early College,"211 Emmons Road | Jackson, MI 49201 | (517) 768-7093",NaN,school_district
489,EHOVE Career Center,"316 W. Mason Rd Milan, OH 44846 Ph 419-499-4663",NaN,school_district
765,NaN,"455 Becker Street | P.O. Box 460 | IGNACIO, CO 81137",Ignacio School District No. 11JT,NaN
1801,NaN,"3301 Beltagh Avenue, Wantagh, NY 11793",Wantagh Union Free School District,NaN
3724,Richard Allen Preparatory Charter School,"2601 S. 58th Street Philadelphia, PA 19143",NaN,school_district


In [1736]:
# let's check its distribution
gpt_df["classification"].value_counts(dropna=False)

classification
NaN                2149
school_district    1704
address              53
Name: count, dtype: int64

In [1737]:
# interestingly, there are no drops
# let's look at the slogan row
gpt_df[gpt_df["title_1"]=="Quality to the Core"]

,title_1,address,school_district,classification
21,Quality to the Core,NaN,Central Dauphin School District,address


In [1738]:
# this is a pretty bad classification
# turns out GPT was using Python under the hood
# I asked GPT to classify manually instead
# here is its results
gpt_df = pd.read_csv("manual_classified_test.csv")
gpt_df.sample(10)

,title_1,address,school_district,classification
2533,NaN,NaN,South Range Local School District,NaN
3812,NaN,"316 W. Naches Ave. Selah, WA 98942",Selah School District,NaN
38,Macon County R-1 Schools,"702 North Missouri  Macon, Missouri 63552  (660) 395-6164",NaN,school_district
2908,NaN,"P.O. Box 8| Greenwood Lake, NY 10925 | p (845) 782-8678 |",Greenwood Lake Union Free School District,NaN
2765,NaN,"176 Walck Road| North Tonawanda, NY 14120 | Ph: 716.807.3500",North Tonawanda City School District,NaN
1547,NaN,"1057 Merrick Avenue, North Merrick, NY 11566 | 516-292-3694",North Merrick Union Free School District NY,NaN
1219,City of Sachse,"3815-B Sachse Road Sachse, TX 75048 | 972.495.1212",NaN,school_district
962,NaN,NaN,Western School District,NaN
1337,NaN,"Culpeper County Administration | 302 N. Main Street | Culpeper VA, 22701 | 540-727-3427 (Phone) | 540-727-3460 (Fax)",NaN,NaN
1593,NaN,NaN,LaBrae Local School District,NaN


In [1739]:
# still terrible, and I realized it still uses python under the hood
# I will start a new session and emphasize not to program
# let's check a few stats first

# number of NA title_1
print(df[df["title_1"].isna()].shape[0])

# number of none NA title_1
print(df[~df["title_1"].isna()].shape[0])

2149
1757


In [1740]:
df.columns

Index(['URL', 'title_1', 'home_website', 'address', 'school_district',
       'contains_phone_number_title_1', 'contains_phone_number_title_2',
       'phone'],
      dtype='object')

In [1741]:
df = df.drop(columns=["contains_phone_number_title_1","contains_phone_number_title_2"])

In [1742]:
# try again
df["index"] = df.index

In [1743]:
# o1 identified these as addresses
print(df.loc[df["index"].isin([155,242, 1343])])
# handle them manually
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "address"] = df["title_1"]
df.loc[df["URL"]=="https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public", "title_1"] = None

# drop the rest
df.loc[df["index"].isin([242, 1343]), "title_1"] = None

                                                          URL  \
155   https://go.boarddocs.com/ny/clintoncsd/Board.nsf/Public   
242          https://go.boarddocs.com/oh/swl/Board.nsf/Public   
1343        https://go.boarddocs.com/pa/karn/Board.nsf/Public   

                              title_1               home_website  \
155   75 Chenango Ave Clinton NY13323       https://www.ccs.edu/   
242          Southwest Licking LSD OH   http://www.swl.k12.oh.us   
1343             1446 Kittanning Pike  https://www.kcasdk12.org/   

                                                                       address  \
155                                                                       None   
242   Address: 927-A South Street, Pataskala, Ohio 43062 | Phone: 740-927-3941   
1343                                                     Karns City, PA  16041   

                      school_district phone  index  
155   Clinton Central School District  None    155  
242                            

In [1744]:
# output the rest
# use prompts/prompt_3.txt
df[~df["title_1"].isna()].to_csv("test.csv", columns=["title_1","index"],index=False)

In [1745]:
# get the o1 classification
gpt_df = pd.read_csv("gpt_results/results_10.csv")

# merge them
df = pd.merge(df, gpt_df, how="left", on="index")


In [1746]:
# get the missing ones
df[(~df["title_1"].isna()) & (df["classification"].isna()) ].to_csv("test-2.csv", columns=["title_1","index"],index=False)

In [1747]:
# sanity check
# all NAs should not have classifications
assert df[(df["title_1"].isna()) & (~df["classification"].isna()) ].shape[0] == 0

# all non-NAs should have classifications
assert df[(~df["title_1"].isna()) & (df["classification"].isna()) ].shape[0] == 0

# the number of classification should be the number of rows on the right
assert gpt_df.shape[0] == df[~df["classification"].isna()].shape[0]

In [1748]:
df["classification"].value_counts()

classification
school_district    1617
drop                 86
unsure               51
Name: count, dtype: int64

In [1749]:
# check the ones that are unsure
df[df["classification"]=="unsure"]

,URL,title_1,home_website,address,school_district,phone,index,classification
26,https://go.boarddocs.com/sd/meade/Board.nsf/Public,"Meade County, South Dakota",http://www.meadecounty.org/,"1300 Sherman Street Ste 222 | Sturgis, SD 57785",NaN,None,26,unsure
152,https://go.boarddocs.com/ks/kta/Board.nsf/Public,Kansas Turnpike Authority,http://www.ksturnpike.com/,"9401 E Kellogg | Wichita, KS 67207 | (316) 682-4537",NaN,None,152,unsure
165,https://go.boarddocs.com/il/vobil/Board.nsf/Public,"The Village of Bourbonnais, Illinois",http://www.villageofbourbonnais.com/,"600 Main St N.W. | Bourbonnais, IL 60914",NaN,None,165,unsure
269,https://go.boarddocs.com/il/naperville/Board.nsf/Public,Naperville Park District,https://www.napervilleparks.org/,"320. W. Jackson Ave. Naperville, IL 60540 | 630-848-5000",NaN,None,269,unsure
303,https://go.boarddocs.com/mn/mboa/Board.nsf/Public,Minnesota Board on Aging,http://www.mn.gov/board-on-aging,"540 Cedar Street St. Paul, MN 55155 | 1-800-882-6262",NaN,None,303,unsure
320,https://go.boarddocs.com/ca/phcd/Board.nsf/Public,Peninsula Health Care District,http://www.peninsulahealthcaredistrict.org,"1819 Trousdale Drive | Burlingame, CA 94010 | (650) 697-6900",NaN,None,320,unsure
609,https://go.boarddocs.com/il/ctpf/Board.nsf/Public,Public School Teachers' Pension and Retirement Fund of Chicago,http://www.ctpf.org,"425 S Financial Place, Suite 1400, Chicago, IL 60605 | 312-604-1400",NaN,None,609,unsure
620,https://go.boarddocs.com/mi/jackson/Board.nsf/Public,"Jackson County, Michigan",http://www.mijackson.org,"120 West Michigan | Jackson, MI 49201 | P: (517) 788-4335 | F: (517) 780-4755",NaN,None,620,unsure
653,https://go.boarddocs.com/fl/sefbhn/Board.nsf/Public,Southeast Florida Behavioral Health Network,http://sefbhn.org/,"8895 N Military Trail, Suite E-102 | Palm Beach Gardens, FL 33410 | (561) 203-2485",NaN,None,653,unsure
697,https://go.boarddocs.com/ok/plib/Board.nsf/Public,Pioneer Library System,http://www.pioneerlibrarysystem.org,"300 Norman Center Court | Norman, OK 73072 | 405.801.4500",NaN,None,697,unsure


In [1750]:
# the unsures all look like valid entities
# and their school_district field is empty
# we will classify them as school_district

df.loc[df["classification"]=="unsure", "classification"] = "school_district"

In [1751]:
# look at those with drops
df.loc[df["classification"]=="drop",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
1818,https://go.boarddocs.com/oh/rockh/Board.nsf/Public,"David Hopper, Superintendent",www.rockhill.org,"Rock Hill Local School District 2325A County Road 26 Ironton, OH 45638",NaN,None,1818,drop
2156,https://go.boarddocs.com/pa/hasdpa/Board.nsf/Public,Inspiring students to discover and achieve their best.,http://www.hasd.us,None,NaN,None,2156,drop
502,https://go.boarddocs.com/oh/hevs/Board.nsf/Public,Hubbard Eagles,www.hubbard.k12.oh.us,"108 Orchard Avenue | Hubbard, Ohio 44425 | 330-534-1921",NaN,None,502,drop
2966,https://go.boarddocs.com/wi/nles/Board.nsf/Public,District Policy Manual,www.northlandlakeschool.com,None,North Lakeland School District,None,2966,drop
1932,https://go.boarddocs.com/wi/ngsd/Board.nsf/Public,NEOLA Board of Education Policy Book,www.ngsd.k12.wi.us,None,New Glarus School District,None,1932,drop


In [1752]:
# drop the ones classified as drops
df.loc[df["classification"]=="drop", "title_1"] = None
df.loc[df["classification"]=="drop", "classification"] = None

In [1753]:
# look at those classified as school_district
df.loc[df["classification"]=="school_district",:].sample(5)

,URL,title_1,home_website,address,school_district,phone,index,classification
3402,https://go.boarddocs.com/mi/copesd/Board.nsf/Public,Cheboygan-Otsego-Presque ESD,http://www.copesd.org,"6065 Learning Lane, Indian River, MI 49749| (231)238-9394",NaN,None,3402,school_district
3857,https://go.boarddocs.com/in/ccsin/Board.nsf/Public,Carmel Clay Schools,http://www1.ccs.k12.in.us,"5201 E. Main Street Carmel, IN 46033 | (317) 844-9961",NaN,None,3857,school_district
1641,https://go.boarddocs.com/mi/byron/Board.nsf/Public,Byron Center Public Schools,http://www.bcpsk12.net,None,NaN,None,1641,school_district
1217,https://go.boarddocs.com/in/fayettecounty/Board.nsf/Public,Fayette County School Corporation,https://www.fayette.k12.in.us/,"1401 Spartan Drive Connersville, IN 47331 | 765.825.2178",NaN,None,1217,school_district
2189,https://go.boarddocs.com/oh/cps/Board.nsf/Public,Cincinnati Public Schools,http://www.cps-k12.org/,2651 Burnet Avenue | Cincinnati OH 45219 | p (513) 363-0000 | tdd (513) 363-0124,NaN,None,2189,school_district


In [1754]:
# move the others to school_district
df.loc[df["classification"]=="school_district", "school_district"] = df["title_1"]
df.loc[df["classification"]=="school_district", "title_1"] = None
df.loc[df["classification"]=="school_district", "classification"] = None

# assert no others
assert df[~df["classification"].isna()].shape[0] == 0
assert df[~df["title_1"].isna()].shape[0] == 0

In [1755]:
# delete these columns
df = df.drop(["classification","title_1"], axis=1)

In [1756]:
# we also try to get the phone number from the address
df['address_contains_phone_number'] = df['address'].apply(contains_phone_number)
df[df["address_contains_phone_number"]].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number
2637,https://go.boarddocs.com/il/fsd157c/Board.nsf/Public,http://www.fsd157c.org/,"10482 Nebraska Street | Frankfort, IL 60423 | (815) 469-5922",Frankfort School District 157-C,None,2637,True
2421,https://go.boarddocs.com/nj/tosd/Board.nsf/Public,http://www.oceanschools.org,"163 Monmouth Road Oakhurst, NJ 07755 | 732-531-5600",Township of Ocean School District,None,2421,True
349,https://go.boarddocs.com/vsba/rcpsva/Board.nsf/Public,http://www.rappahannockschools.us,"6 School House Road, Washington, VA 22747 Phone: 540-227-0023",Rappahannock County Public Schools,None,349,True
3597,https://go.boarddocs.com/in/lpcsc/Board.nsf/Public,http://www.lpcsc.k12.in.us,"1000 Harrison Street | La Porte, Indiana 46350 | Ph: 219-362-7056 Fx: 219-324-9347",La Porte Community School Corporation,None,3597,True
3813,https://go.boarddocs.com/mi/chavezacad/Board.nsf/Public,https://www.chavezacademy.com/,"4100 Martin Street | Detroit, MI 48210 | 313-361-8689",César Chávez Academy,None,3813,True


In [1757]:
df[df["address_contains_phone_number"]].shape

(2185, 7)

In [1758]:
# extract it
# but before that, check if conflicts
df[df["address_contains_phone_number"] & (~df["phone"].isna())].shape

(0, 7)

In [1759]:
# great! no conflicts
phone_pattern = re.compile(r'(\(?\b\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b)')
df["phone_extracted_from_address"] = df["address"].str.extract(phone_pattern)

In [1760]:
df[~df["phone_extracted_from_address"].isna()].sample(5)

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address
1633,https://go.boarddocs.com/va/iwcs/Board.nsf/Public,https://www.iwcs.k12.va.us,"820 W MAIN ST., SMITHFIELD, VA 23430 | 757-357-4393",Isle of Wight County Schools VA,None,1633,True,757-357-4393
2283,https://go.boarddocs.com/ca/ogsd/Board.nsf/Public,https://www.ogsd.net,"6578 Santa Teresa Blvd, San Jose, CA 95119 | (408) 227-8300",Oak Grove School District,None,2283,True,(408) 227-8300
1035,https://go.boarddocs.com/wy/fcsd38/Board.nsf/Public,www.arapahoeschools.com,"445 Little Wind River Bottom Road | Arapahoe, WY 82510 | P: 307.856.9333 | F: 307.857.4327",Fremont County School District #38,None,1035,True,307.856.9333
1184,https://go.boarddocs.com/oh/delawareareacc/Board.nsf/Public,http://www.DelawareAreaCC.org/,"4565 Columbus Pike | Delaware, OH 43015 | 740-548-0708",Delaware Area Career Center,None,1184,True,740-548-0708
2224,https://go.boarddocs.com/wa/ohsd/Board.nsf/Public,http://www.ohsd.net,350 S. Oak Harbor St. | Oak Harbor WA 98277 | Phone: 360.279.5000,Oak Harbor Public Schools,None,2224,True,360.279.5000


In [1761]:
# double check no clashes
df[(~df["phone_extracted_from_address"].isna()) & (~df["phone"].isna())]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address


In [1762]:
# write in
df.loc[~df["phone_extracted_from_address"].isna(), "phone"] = df["phone_extracted_from_address"]
df.loc[~df["phone_extracted_from_address"].isna(), "phone_extracted_from_address"] = None

In [1763]:
# let's clean it
# extract all numbers and make sure there are 10

df['extracted_phone_numbers'] = None
df['extracted_phone_numbers'] = df['phone'].astype(str).str.replace(r'[^\d]', '', regex=True)

In [1764]:
# check their length
df["phone_length"] = df["extracted_phone_numbers"].apply(lambda x: len(x))
df["phone_length"].value_counts()

phone_length
10    2216
0     1684
20       4
12       1
30       1
Name: count, dtype: int64

In [1765]:
# check those that are not 10 or 0
df.loc[(df["phone_length"]!=10) & (df["phone_length"]!=0)]

,URL,home_website,address,school_district,phone,index,address_contains_phone_number,phone_extracted_from_address,extracted_phone_numbers,phone_length
624,https://go.boarddocs.com/in/triton/Board.nsf/Public,http://www.triton.k12.in.us,None,Triton School Corporation,Home of the Trojans/574-342-2255/www.triton.k12.in.us,624,False,NaN,574342225512,12
1199,https://go.boarddocs.com/wi/sdathen/Board.nsf/Public,https://www.athens1.org,"School District of Athens, 601 West Limits Road, Athens, WI 54411",NaN,Phone: 715-257-7511 Fax: 715-257-7502,1199,False,NaN,71525775117152577502,20
2173,https://go.boarddocs.com/ks/usd230/Board.nsf/Public,http://www.usd230.org,"Spring Hill School District | 17640 W. 199th Street, Spring Hill, Kansas 66083",NaN,Info: (913) 592-7272 | T: (913) 592-7200 | F: (913) 592-7270,2173,False,NaN,913592727291359272009135927270,30
2538,https://go.boarddocs.com/wi/campsd/Board.nsf/Public,www.csd.k12.wi.us,"327 N. Fond du Lac Ave. Campbellsport, WI 53010",NaN,(920) 533-8381 | Fax (920) 533 -5726,2538,False,NaN,92053383819205335726,20
2586,https://go.boarddocs.com/oh/cevsdoh/Board.nsf/Public,www.carrollton.k12.oh.us,"Carrollton Exempted Village School District, - 205 Scio Road S.W., - Carrollton, OH 44615",NaN,"Phone: 330-627-2181, - Fax: 330-627-2182",2586,False,NaN,33062721813306272182,20
2811,https://go.boarddocs.com/id/nsd131/Board.nsf/Public,http://www.nsd131.org,"619 S. Canyon St | Nampa, ID 83686",NaN,(208) 468-4600 Fax: (208) 468-4638,2811,False,NaN,20846846002084684638,20


In [1766]:
# ok, we can just take the top 10 numbers
df["extracted_phone_numbers"] = df["extracted_phone_numbers"].apply(lambda x: x[:10] if x else None)

In [1767]:
# put them in a nice format
def format_phone_numbers(number):
    if not number:
        return None
    # Convert number to string for easy slicing
    num_str = str(number)
    if len(num_str) == 10:  # Ensure it's a 10-digit number
        formatted = f"({num_str[:3]}) {num_str[3:6]}-{num_str[6:]}"
        return formatted
    else:
        raise Exception("Invalid number")

df["phone"] = df["extracted_phone_numbers"].apply(format_phone_numbers)

In [1768]:
# clean up the columns
df = df.loc[:,["URL","home_website","address","school_district","phone"]]
df.sample(5)

,URL,home_website,address,school_district,phone
1247,https://go.boarddocs.com/ny/saratogaschools/Board.nsf/Public,https://www.saratogaschools.org/,"3 Blue Streak Boulevard Saratoga Springs, NY 12866",Saratoga Springs City School District,None
1747,https://go.boarddocs.com/ca/laguna/Board.nsf/Public,https://www.lbusd.org,"550 Blumont St. Laguna Beach, CA 92651 | 949-497-7700",Laguna Beach Unified School District,(949) 497-7700
3851,https://go.boarddocs.com/wa/wsvsd/Board.nsf/Public,https://www.wsvsd.org/,"170 NW Lincoln Street White Salmon, Washington WA 98672",White Salmon Valley School District,None
586,https://go.boarddocs.com/il/sd54/Board.nsf/Public,http://sd54.org/,"524 E. Schaumburg Road | Schaumburg, IL 60194 | Phone: (847) 357-5000 Fax: (847) 357-5006",School District 54,(847) 357-5000
1276,https://go.boarddocs.com/vsba/roecnty/Board.nsf/Public,http://www.rcs.k12.va.us/,"5937 Cove Road • Roanoke, VA 24019 • Phone: 540-562-3772 • Fax: 540-562-3993",Roanoke County Public Schools,(540) 562-3772


In [1769]:
# finally, add a column indicate if the row is likely an actual school_district
# here is Tom's quote
"""
However, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.
"""

'\nHowever, please add a column to the spreadsheet noting whether the words “school”, “academy”, “district” or “education” appear in the “address” or “school_district” columns.   That way, we can go through later and weed out the other organizations.\n'

In [1770]:
likely_school_district_pattern = r"school|academy|district|education"
df["contain_school_district_keywords"] = df["address"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) | df["school_district"].str.contains(likely_school_district_pattern, case=False, regex=True, na=False) 

In [1771]:
import validators

df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df["valid_home_website"].value_counts()

valid_home_website
True     3792
False     114
Name: count, dtype: int64

In [1772]:
# non-null but invalid home website
weird_websites = df[(~df["valid_home_website"]) & (~df["home_website"].isna())]
weird_websites.shape

(61, 7)

In [1773]:
weird_websites

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,https://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,https://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
...,...,...,...,...,...,...,...
3679,https://go.boarddocs.com/pa/iu08/Board.nsf/Public,http://www.iu08.org,None,Appalachia Intermediate Unit 8,None,False,False
3741,https://go.boarddocs.com/mi/wpcs/Board.nsf/Public,http:/www.wpcschools.org,None,White Pigeon Community Schools,None,True,False
3832,https://go.boarddocs.com/pa/fleb/Board.nsf/Public,http://www.fortleboeuf.net,None,Fort LeBoeuf School District,None,True,False
3876,https://go.boarddocs.com/pa/avnw/Board.nsf/Public,http://www.avonworth.k12.pa.us/,"258 Josephs Lane Pittsburgh, PA 15237",Avonworth School District,None,True,False


In [1774]:
# replace \\
idx = (~df["valid_home_website"]) & (~df["home_website"].isna())
df.loc[idx,"home_website"] = df.loc[:,"home_website"].str.replace("\\","/")

In [1775]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,https://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,https://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
569,https://go.boarddocs.com/wi/fond/Board.nsf/Public,http:/www.fonddulac.k12.wi.us,"72 West Ninth Street, Fond du Lac, WI 54935 (920) 906-6500",Fond du Lac School District,(920) 906-6500,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False


In [1776]:
# fix the typos
pattern = r"https?:?/?/(?=[A-Za-z])"
idx = (~df["valid_home_website"]) & (~df["home_website"].isna())
df.loc[idx,"home_website"] = df.loc[:,"home_website"].str.replace(pattern,"http://", regex=True)

In [1777]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,http://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
173,https://go.boarddocs.com/in/wesdel/Board.nsf/Public,http://www.wes-del.k12.in us,None,Wes-Del Community Schools,None,True,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,http://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,http://www.laurel.k12.pa.us,None,Laurel School District,None,True,False


In [1778]:
# fix one
df.loc[df["URL"]=="https://go.boarddocs.com/in/wesdel/Board.nsf/Public", "home_website"] = "http://www.wes-del.k12.in.us"

# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website
118,https://go.boarddocs.com/nj/pctvs/Board.nsf/Public,http://go.boarddocs.com/nj/pctvs/Board.nsf/Private?open&login#,"45 Reinhardt Road, Wayne, NJ 07470",Passaic County Technical-Vocational Institute,None,False,False
199,https://go.boarddocs.com/pa/bthl/Board.nsf/Public,http://www.bpsd.org,"301 Church Road, Bethel Park, PA 15102",Bethel Park School District,None,True,False
210,https://go.boarddocs.com/ny/elmont/Board.nsf/Public,http://www.elmontschools.org/,"135 Elmont Road | Elmont, NY 11003-1635 | 516-326-5500 | f 516-326-5574",Elmont Union Free School District,(516) 326-5500,True,False
301,https://go.boarddocs.com/wi/lakelanduhs/Board.nsf/Public,http://www.boarddocs.com/wi/lakelanduhs/ Board.nsf/Public?open&id=policies,None,Lakeland Union High School District,None,True,False
434,https://go.boarddocs.com/wi/ncraw/Board.nsf/Public,www.northcrawford.com,"47050 County Road X | Soldiers Grove, WI 54655 | (608) 735-4318",North Crawford School District,(608) 735-4318,True,False
473,https://go.boarddocs.com/pa/bldw/Board.nsf/Public,http://www.bwschools.net/,None,Baldwin-Whitehall School District,None,True,False
611,https://go.boarddocs.com/pa/bgbf/Board.nsf/Public,http://www.tigerweb.org/,None,Big Beaver Falls School District,None,True,False
639,https://go.boarddocs.com/oh/young/Board.nsf/Public,http://www.ycsd.org,None,Youngstown City Schools District,None,True,False
673,https://go.boarddocs.com/pa/laur/Board.nsf/Public,http://www.laurel.k12.pa.us,None,Laurel School District,None,True,False
694,https://go.boarddocs.com/pa/eriesd/Board.nsf/Public,http://www.boarddocs.com/pa/eriesd/Board.nsf/Private?open&login,"148 West 21st Street | Erie, Pennsylvania 16502 | 814-874-6000","The School District of the City of Erie, Pennsylvania",(814) 874-6000,True,False


In [1779]:
# we don't care about the rest of the URL after the first singular /
# and we also don't care about spaces
# their spaces should be after / though

df["spaces_before_slash"] = df["home_website"].apply(lambda x: ' ' in str(x).replace("//","").split('/', 1)[0] if x else False)
df["spaces_before_slash"].value_counts()

spaces_before_slash
False    3884
True       22
Name: count, dtype: int64

In [1780]:
# chop off the rest of the URL if they have spaces
df.loc[:,"home_website"] = df["home_website"].apply(lambda x: str(x).split(' ', 1)[0] if x else None)

# also just keep the things before the slash
single_slash = r"(?<!/)/(?!/)"
df.loc[:,"home_website"] = df["home_website"].apply(lambda x: re.split(single_slash, str(x))[0] if x else None)

In [1781]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash
232,https://go.boarddocs.com/pa/camb/Board.nsf/Public,nan,None,NaN,None,False,False,False
443,https://go.boarddocs.com/oh/oakhil/Board.nsf/Public,nan,None,NaN,None,False,False,False
465,https://go.boarddocs.com/oh/warrenoh/Board.nsf/Public,nan,None,NaN,None,False,False,False
468,https://go.boarddocs.com/pa/marp/Board.nsf/Public,nan,None,Marple Newtown School District,None,True,False,False
533,https://go.boarddocs.com/oh/meigs/Board.nsf/Public,nan,None,NaN,None,False,False,False
552,https://go.boarddocs.com/pa/sola/Board.nsf/Public,nan,None,NaN,None,False,False,False
590,https://go.boarddocs.com/in/lanesville/Board.nsf/Public,nan,None,Lanesville Comunity School Corporation,None,True,False,False
784,https://go.boarddocs.com/wi/sbschools/Board.nsf/Public,nan,None,Stanley-Boyd Area Schools,None,True,False,False
884,https://go.boarddocs.com/oh/vwcs/Board.nsf/Public,nan,None,Van Wert City Schools,None,True,False,False
996,https://go.boarddocs.com/ca/lmsv/Board.nsf/Public,nan,None,NaN,None,False,False,False


In [1782]:
df.loc[df["home_website"] == "nan", "home_website"] = None

In [1783]:
# fix manual
df.loc[df["URL"]=="https://go.boarddocs.com/pa/ojrsd/Board.nsf/Public", "home_website"] = "http://www.ojrsd.com"
df.loc[df["URL"]=="https://go.boarddocs.com/mi/sutt/Board.nsf/Public", "home_website"] = "http://www.suttonsbayschools.com"

In [1784]:
# non-null but invalid home website
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True or validators.url("http://" + str(x)) == True)
df[(~df["valid_home_website"]) & (~df["home_website"].isna())]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash


In [1785]:
# add http
df["valid_home_website"] = df["home_website"].apply(lambda x: validators.url(x) == True)
df.loc[~df["valid_home_website"], "home_website"] = "http://" + df["home_website"]

In [1786]:
# non-null but invalid home website
df["valid_URL"] = df["URL"].apply(lambda x: validators.url(x) == True)
df["valid_URL"].value_counts()

valid_URL
True     3905
False       1
Name: count, dtype: int64

In [1787]:
df[~df["valid_URL"]]

,URL,home_website,address,school_district,phone,contain_school_district_keywords,valid_home_website,spaces_before_slash,valid_URL
3553,NaN,NaN,None,NaN,None,False,False,False,False


In [1788]:
idx = df[~df["valid_URL"]].index
df = df.drop(idx,axis=0)

In [1789]:
# non-null but invalid home website
df["valid_URL"] = df["URL"].apply(lambda x: validators.url(x) == True)
df["valid_URL"].value_counts()

valid_URL
True    3905
Name: count, dtype: int64

In [1790]:
# check those that link to boarddocs
df["boarddocs_as_home_website"] = df["home_website"].str.contains("boarddocs", na=False)
df["boarddocs_as_home_website"].value_counts()

boarddocs_as_home_website
False    3873
True       32
Name: count, dtype: int64

In [1791]:
# just remove these
df.loc[df["boarddocs_as_home_website"], "home_website"] = None

In [1792]:
# ignore case in URL
df["url"] = df["URL"].str.lower()
df = df[~df["url"].duplicated(keep="first")]
df = df.drop("url", axis=1)

In [1793]:
# output the result
df.to_csv("release/deliverable_1.csv", index=False, columns=["URL","school_district","address","home_website","phone"])

In [1794]:
# get more stats
df.shape

(3896, 10)

In [1796]:
# trim all whitespace before output
df["URL"] = df["URL"].str.strip()
df["school_district"] = df["school_district"].str.strip()
df["address"] = df["address"].str.strip()
df["home_website"] = df["home_website"].str.strip()
df["phone"] = df["phone"].str.strip()

In [1797]:
len(df["URL"].unique())

3896

In [1798]:
# home website
(~df["home_website"].isna()).value_counts()

home_website
True     3811
False      85
Name: count, dtype: int64

In [1799]:
# home website (in percentages)
(~df["home_website"].isna()).value_counts() / df.shape[0] * 100

home_website
True     97.818275
False     2.181725
Name: count, dtype: float64

In [1800]:
# phone numbers
(~df["phone"].isna()).value_counts()

phone
True     2215
False    1681
Name: count, dtype: int64

In [1801]:
# phone (in percentages)
(~df["phone"].isna()).value_counts() / df.shape[0] * 100

phone
True     56.853183
False    43.146817
Name: count, dtype: float64

In [1802]:
# phone or website
((~df["phone"].isna()) | (~df["home_website"].isna())).value_counts()

True     3835
False      61
Name: count, dtype: int64

In [1803]:
# phone or website (percentages)
((~df["phone"].isna()) | (~df["home_website"].isna())).value_counts()/ df.shape[0] * 100

True     98.434292
False     1.565708
Name: count, dtype: float64

In [1804]:
# school_district
(~df["school_district"].isna()).value_counts()

school_district
True     3632
False     264
Name: count, dtype: int64

In [1805]:
# school_district (in percentages)
(~df["school_district"].isna()).value_counts() / df.shape[0] * 100

school_district
True     93.223819
False     6.776181
Name: count, dtype: float64

In [1806]:
# address
(~df["address"].isna()).value_counts()

address
True     2911
False     985
Name: count, dtype: int64

In [1807]:
# address (in percentages)
(~df["address"].isna()).value_counts() / df.shape[0] * 100

address
True     74.717659
False    25.282341
Name: count, dtype: float64

In [1808]:
# get the stats
df["contain_school_district_keywords"].value_counts()

contain_school_district_keywords
True     3399
False     497
Name: count, dtype: int64

In [1809]:
# address (in percentages)
df["contain_school_district_keywords"].value_counts() / df.shape[0] * 100

contain_school_district_keywords
True     87.243326
False    12.756674
Name: count, dtype: float64